# OpenCV 编程入门 - Python 版 - 4.5.1

书中很多例子都改编自 [OpenCV 官方文档](https://docs.opencv.org/4.5.1/)，opencv-python 中部分函数与 C++ 版本有些许差别，遇到问题时建议多查阅相关说明和教程：

[OpenCV C++ 版本教程](https://docs.opencv.org/4.5.1/d9/df8/tutorial_root.html)

[opencv-python 教程](https://docs.opencv.org/4.5.1/d6/d00/tutorial_py_root.html)

In [2]:
# 依赖环境 - python 3.8
import argparse
import copy
import numpy as np

import matplotlib.pyplot as plt
# 在独立窗口显示动态图（可能的报错原因是未安装依赖包：pip install PyQt5）
%matplotlib qt5
"""
# 在网页内显示静态图
%matplotlib inline
# 在独立窗口显示动态图
%matplotlib auto
"""

import cv2
# 当前使用的 OpenCV 版本为 4.5.1
print("当前使用的 OpenCV 版本为", cv2.__version__)


当前使用的 OpenCV 版本为 4.5.1


## 第一部分 - 快速上手 OpenCV

### 第一章 - 邂逅 OpenCV

#### 1.5 - 快速上手 OpenCV 图像处理

注意：在像素坐标系中，坐标原点在图像左上角。

OpenCV 中的 (x, y) 表示图像矩阵的 列、行。

C++ 中 img.size() 返回的 (x, y) 表示图像矩阵的 列、行。

Python 中 img.shape 返回的 (x, y, d) 表示图像矩阵的 **行、列**、通道数

In [3]:
# 图像显示
img = cv2.imread("./image/1_HelloOpenCV.jpg")   # 载入图像，第二个参数含义：0 灰度图，1 彩色图，2 任意深度图，默认为 1
cv2.imshow("Showing the image", img)  # 显示图像
cv2.waitKey(0)  # 等待任意按键按下
cv2.destroyAllWindows()

In [4]:
# 图像腐蚀
img = cv2.imread("./image/3_ImageErode.jpg")
element = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
dst = cv2.erode(img, element)
result = np.hstack([img, dst])
cv2.imshow("Eroding the image", result)
cv2.waitKey()
cv2.destroyAllWindows()

In [5]:
# 图像模糊
img = cv2.imread("./image/4_BlurImage.jpg")
dst = cv2.blur(img, (7, 7))
result = np.hstack([img, dst])
cv2.imshow("Blurring the image", result)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
# canny 边缘检测
img = cv2.imread("./image/5_Canny.jpg")
cv2.imshow("Original image: ", img)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # 转为灰度图
img_gray_blur = cv2.blur(img_gray, (3, 3))    # 降噪
img_edge = cv2.Canny(img_gray_blur, 3, 9, 3)
result = np.hstack([img_gray, img_gray_blur, img_edge])
cv2.imshow("Canny edge detection", result)

cv2.waitKey(5000)
cv2.destroyAllWindows()

#### 1.6 - OpenCv 视频操作基础

In [7]:
# 读取并播放视频
capture = cv2.VideoCapture("./video/6_PlayVideo.avi")
while capture.isOpened():
    ret, frame = capture.read()
    if not ret: # 正确读取帧时 ret 为真
        print("Cannot receive frame (stream end?). Exiting ...")
        break
    cv2.imshow("frame", frame)
    if cv2.waitKey(30) == ord('q'): # 延时 30ms
        break   # 若按下 q 则退出视频播放
capture.release()
cv2.destroyAllWindows()

In [8]:
# 调用摄像头采集图像
capture = cv2.VideoCapture(0)
while capture.isOpened():
    ret, frame = capture.read()
    if not ret:
        print("Cannot receive frame (stream end?). Exiting ...")
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_blur = cv2.blur(frame_gray, (7, 7))
    frame_edge = cv2.Canny(frame_blur, 3, 9, 3)
    cv2.imshow("frame", frame_edge)
    if cv2.waitKey(30) == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

### 第三章 - HighGUI 图形用户界面初步

#### 3.2 - 滑动条的创建和使用 

In [10]:
# 创建滑动条
alpha_slider_max = 100  # Alpha 值的最大值
alpha_slider_init = 70  # 设置滑动条初值
title_window = 'Linear Blend'


# def on_trackbar(val):
def on_trackbar(_):
    val = cv2.getTrackbarPos(trackbar_name, title_window)   # 获取当前轨迹条的位置
    alpha = float(val / alpha_slider_max)  # 求出当前 alpha 值相对于最大值的比例
    beta = (1.0 - alpha)    # 则 bate 值为 1 减去 alpha 值
    dst = cv2.addWeighted(src1, alpha, src2, beta, 0.0) # 根据 alpha 和 beta 值进行线性混合
    cv2.imshow(title_window, dst)


parser = argparse.ArgumentParser(
    description='Code for Adding a Trackbar to our applications tutorial.')
parser.add_argument(
    '--input1', help='Path to the first input image.', default="./image/17_1.jpg")
parser.add_argument(
    '--input2', help='Path to the second input image.', default="./image/17_2.jpg")
args = parser.parse_args(args=[])
src1 = cv2.imread(cv2.samples.findFile(args.input1))
src2 = cv2.imread(cv2.samples.findFile(args.input2))
if src1 is None:
    print('Could not open or find the image: ', args.input1)
    exit(0)
if src2 is None:
    print('Could not open or find the image: ', args.input2)
    exit(0)

cv2.namedWindow(title_window)   # 创建窗体
trackbar_name = 'Alpha %d' % alpha_slider_max   # 在创建的窗体中创建一个滑动条控件
cv2.createTrackbar(trackbar_name, title_window, alpha_slider_init, alpha_slider_max, on_trackbar)

# Show some stuff
on_trackbar(0)
# Wait until user press some key
cv2.waitKey()

-1

#### 3.3 - 鼠标操作

In [12]:
# -*- coding: utf-8 -*-
"""
版权声明：本文为博主原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接和本声明。
本文链接：https://blog.csdn.net/bby1987/article/details/107302398
"""
WIN_NAME = 'draw_rect'


class Rect(object):
    def __init__(self):
        self.tl = (0, 0)    # top-left point
        self.br = (0, 0)    # bottom-right point

    def regularize(self):
        # When moving from the bottom-right to the top-left, you should make sure tl = TopLeft point, br = BottomRight point
        pt1 = (min(self.tl[0], self.br[0]), min(self.tl[1], self.br[1]))
        pt2 = (max(self.tl[0], self.br[0]), max(self.tl[1], self.br[1]))
        self.tl = pt1
        self.br = pt2


class DrawRects(object):
    def __init__(self, image, color, thickness=1):
        self.original_image = image
        self.image_for_show = image.copy()
        self.color = color
        self.thickness = thickness
        self.rects = []
        self.current_rect = Rect()
        self.left_button_down = False

    @staticmethod
    def __clip(value, low, high):
        # clip value between low and high
        output = max(value, low)
        output = min(output, high)
        return output

    def shrink_point(self, x, y):
        # shrink point (x, y) to inside image_for_show, that is, limit the drawn rectangle to the window
        height, width = self.image_for_show.shape[0:2]
        x_shrink = self.__clip(x, 0, width)
        y_shrink = self.__clip(y, 0, height)
        return (x_shrink, y_shrink)

    def append(self):
        # add a rect to rects list
        self.rects.append(copy.deepcopy(self.current_rect))

    def pop(self):
        # pop a rect from rects list
        rect = Rect()
        if self.rects:
            rect = self.rects.pop()
        return rect

    def reset_image(self):
        # reset image_for_show using original image
        self.image_for_show = self.original_image.copy()

    def draw(self):
        # draw a series of rects on image_for_show
        for rect in self.rects:
            cv2.rectangle(self.image_for_show, rect.tl, rect.br,
                          color=self.color, thickness=self.thickness)

    def draw_current_rect(self):
        # draw current rect on image_for_show
        cv2.rectangle(self.image_for_show,
                      self.current_rect.tl, self.current_rect.br,
                      color=self.color, thickness=self.thickness)


def onmouse_draw_rect(event, x, y, flags, draw_rects):
    if event == cv2.EVENT_LBUTTONDOWN:
        # pick first point of rect
        print('pt1: x = %d, y = %d' % (x, y))
        draw_rects.left_button_down = True
        draw_rects.current_rect.tl = (x, y)
    if draw_rects.left_button_down and event == cv2.EVENT_MOUSEMOVE:
        # pick second point of rect and draw current rect
        draw_rects.current_rect.br = draw_rects.shrink_point(x, y)
        draw_rects.reset_image()
        draw_rects.draw()
        draw_rects.draw_current_rect()
    if event == cv2.EVENT_LBUTTONUP:
        # finish drawing current rect and append it to rects list
        draw_rects.left_button_down = False
        draw_rects.current_rect.br = draw_rects.shrink_point(x, y)
        print('pt2: x = %d, y = %d' % (draw_rects.current_rect.br[0],
                                       draw_rects.current_rect.br[1]))
        draw_rects.current_rect.regularize()
        draw_rects.append()
    if (not draw_rects.left_button_down) and event == cv2.EVENT_RBUTTONDOWN:
        # pop the last rect in rects list
        draw_rects.pop()
        draw_rects.reset_image()
        draw_rects.draw()


if __name__ == '__main__':
    image = np.zeros((600, 800, 3), np.uint8)   # background
    draw_rects = DrawRects(image, (0, 255, 0), 2)
    cv2.namedWindow(WIN_NAME)
    cv2.setMouseCallback(WIN_NAME, onmouse_draw_rect, draw_rects)
    while True:
        cv2.imshow(WIN_NAME, draw_rects.image_for_show)
        key = cv2.waitKey(30)
        if key == 27:  # ESC
            break
    cv2.destroyAllWindows()


pt1: x = 171, y = 188
pt2: x = 514, y = 384


## 第二部分 - 初探 core 组件

### 第四章 - OpenCV 数据结构与基本绘图

#### 4.3 - 基本图形的绘制

In [13]:
# 基本图形的绘制
W = 400


def my_ellipse(img, angle):
    thickness = 2
    line_type = 8
    cv2.ellipse(img,
               (W // 2, W // 2),    # 椭圆中心点
               (W // 4, W // 16),   # 大小位于该矩形框内
               angle,               # 椭圆旋转角度
               0,                   # 扩展的弧度的变化范围
               360,
               (255, 0, 0),         # 图形颜色
               thickness,           # 线宽 
               line_type)           # 线型 - 联通


def my_filled_circle(img, center):
    thickness = -1
    line_type = 8
    cv2.circle(img,
              center,
              W // 32,  # 圆的半径
              (0, 0, 255),
              thickness,
              line_type)

# 绘制凹多边形
def my_polygon(img):
    line_type = 8
    # Create some points
    ppt = np.array([[W / 4, 7 * W / 8], [3 * W / 4, 7 * W / 8],
                    [3 * W / 4, 13 * W / 16], [11 * W / 16, 13 * W / 16],
                    [19 * W / 32, 3 * W / 8], [3 * W / 4, 3 * W / 8],
                    [3 * W / 4, W / 8], [26 * W / 40, W / 8],
                    [26 * W / 40, W / 4], [22 * W / 40, W / 4],
                    [22 * W / 40, W / 8], [18 * W / 40, W / 8],
                    [18 * W / 40, W / 4], [14 * W / 40, W / 4],
                    [14 * W / 40, W / 8], [W / 4, W / 8],
                    [W / 4, 3 * W / 8], [13 * W / 32, 3 * W / 8],
                    [5 * W / 16, 13 * W / 16], [W / 4, 13 * W / 16]], np.int32)
    ppt = ppt.reshape((-1, 1, 2))   # 一行两列，每行对应一个顶点；第三个维度由矩阵确定
    cv2.fillPoly(img, [ppt], (255, 255, 255), line_type)
    # Only drawind the lines would be:
    # cv2.polylines(img, [ppt], True, (255, 0, 255), line_type)   # 第三个参数确定多边形是否闭合


def my_line(img, start, end):
    thickness = 2
    line_type = 8
    cv2.line(img,
            start,
            end,
            (0, 0, 0),
            thickness,
            line_type)


atom_window = "Drawing 1: Atom"
rook_window = "Drawing 2: Rook"
# Create black empty images
size = W, W, 3
atom_image = np.zeros(size, dtype=np.uint8)
rook_image = np.zeros(size, dtype=np.uint8)
# 1.a. Creating ellipses
my_ellipse(atom_image, 90)
my_ellipse(atom_image, 0)
my_ellipse(atom_image, 45)
my_ellipse(atom_image, -45)
# 1.b. Creating circles
my_filled_circle(atom_image, (W // 2, W // 2))
# 2. Draw a rook
# ------------------
# 2.a. Create a convex polygon
my_polygon(rook_image)
cv2.rectangle(rook_image,
             (0, 7 * W // 8),   # 矩形左上角顶点
             (W, W),            # 矩形右下角顶点，注意原点在窗口的左上角，(x,y) 对应矩阵的列和行
             (0, 255, 255),
             -1,
             8)
#  2.c. Create a few lines
my_line(rook_image, (0, 15 * W // 16), (W, 15 * W // 16))
my_line(rook_image, (W // 4, 7 * W // 8), (W // 4, W))
my_line(rook_image, (W // 2, 7 * W // 8), (W // 2, W))
my_line(rook_image, (3 * W // 4, 7 * W // 8), (3 * W // 4, W))
cv2.imshow(atom_window, atom_image)
cv2.moveWindow(atom_window, 0, 200)
cv2.imshow(rook_window, rook_image)
cv2.moveWindow(rook_window, W, 200)
cv2.waitKey(0)
cv2.destroyAllWindows()


### 第五章 - core 组件进阶

#### 5.1 - 访问图像中的像素

In [14]:
# 计时函数
freq = cv2.getTickFrequency()  # 返回 CPU 一秒钟所走的时钟周期数
print("%d times" % freq)

time = cv2.getTickCount()   # 返回 CPU 自某个事件以来走过的时钟周期数
# ...procedure...
time = (cv2.getTickCount() - time) / cv2.getTickFrequency() # 程序运行时间
print("%f s" % time)

10000000 times
0.000056 s


#### 5.2 - ROI 区域图像叠加 & 图像混合

In [15]:
# 初级图像混合 -> ROI(Region of Interest) + addWeighted
img = cv2.imread("./image/26_dota_jugg.jpg")
logo = cv2.imread("./image/26_dota_logo.jpg")
img_roi = img[200:200+logo.shape[0], 550:550+logo.shape[1]] # 切片
cv2.addWeighted(img_roi, 0.5, logo, 0.3, 0, img_roi)
# img_roi = cv2.addWeighted(img_roi, 0.5, logo, 0.3, 0) # 与上一行的写法等价；当 dst 未指定时，使用该行代码
cv2.imshow("the example of region linear blend", img)
cv2.waitKey()
cv2.destroyAllWindows()

#### 5.3 - 分离颜色通道、多通道图像混合

In [16]:
# 颜色通道分离 split & 多通道图像混合 merge
img = cv2.imread("./image/26_dota_jugg.jpg")
blue, green, red = cv2.split(img)
zeros = np.zeros(img.shape[:2], dtype="uint8")
cv2.imshow("split the image to blue channel", cv2.merge([blue, zeros, zeros]))

# split()、merge() 和 cvtColor() 都是 mixChannels() 的一部分
# mixChannels() 的作用是将输入参数的某通道复制给输出参数的特定通道
rgba = np.zeros([500, 500, 4], dtype="uint8")
for i in range(4):
    rgba[:,:,i] = 50 + i*50
bgr = np.zeros([rgba.shape[0], rgba.shape[1], 3], dtype="uint8")
alpha = np.zeros([rgba.shape[0], rgba.shape[1]], dtype="uint8")

# 将一个 4 通道的 RGBA 图像转化为 3 通道的 BGR 和一个单通道的 Alpha 图像
from_to = [0,2, 1,1, 2,0, 3,3]
cv2.mixChannels(src=[rgba], dst=[bgr, alpha], fromTo=from_to)
cv2.imshow("rgba", rgba)
cv2.imshow("bgr", bgr)
cv2.imshow("alpha", alpha)

cv2.waitKey()
cv2.destroyAllWindows()

#### 5.4 - 图像对比度、亮度值调整

In [17]:
# 图像对比度和亮度值调整
def on_contrast_and_bright(_):
    contrast = cv2.getTrackbarPos("Contrast", "result")
    bright = cv2.getTrackbarPos("Bright", "result")
    
    mat_contrast = np.ones(img.shape, dtype=img.dtype) * contrast * 0.01
    mat_bright = np.ones(img.shape, dtype=img.dtype) * bright
    """
    cv2.add/multiply 与 saturate_cast 模板函数类似，用于溢出保护
    cv2.add/multiply 中两个矩阵的尺寸应相同，且数据类型不一致时，需要声明返回图像的数据类型
    cv2 的 dtype 与 numpy 不同，必须使用标识符（或对应的整数）    
    """
    dst = cv2.add(
        cv2.multiply(mat_contrast, img, dtype=cv2.CV_8UC3),
        mat_bright)
    
    result = np.hstack([img, dst])
    cv2.imshow("result", result)

img = cv2.imread("./image/27_ChangeContrastAndBright.jpg")
contrast_init = 80
bright_init = 80

cv2.namedWindow("result")
cv2.createTrackbar("Contrast", "result", contrast_init,
                   300, on_contrast_and_bright)
cv2.createTrackbar("Bright", "result", bright_init,
                   200, on_contrast_and_bright)

on_contrast_and_bright(0)

cv2.waitKey()
cv2.destroyAllWindows()


#### 5.5 - 离散傅里叶变换

In [18]:
# 离散傅里叶变换 & 逆变换
img = cv2.imread('./image/28_DFT.jpg', 0)   # DFT 要求单通道或双通道图像

# Performance Optimization of DFT
rows, cols = img.shape
nrows = cv2.getOptimalDFTSize(rows)
ncols = cv2.getOptimalDFTSize(cols)
right = ncols - cols
bottom = nrows - rows
bordertype = cv2.BORDER_CONSTANT  # just to avoid line breakup in PDF file
nimg = cv2.copyMakeBorder(img, 0, bottom, 0, right, bordertype, value=0)

dft = cv2.dft(np.float32(nimg), flags=cv2.DFT_COMPLEX_OUTPUT)    # 就地操作(in-place)；频域值范围远大于空间值，需要储存在 float 格式中；分为实部和虚部
dft_shift = np.fft.fftshift(dft)  # 将原点平移到中心
magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))  # 频谱图；首先将复数转换为幅值，再取对数进行尺度缩小

crow, ccol = rows//2, cols//2
# create a mask first, center square is 1, remaining all zeros
mask = np.zeros((nrows, ncols, 2), np.uint8)
mask[crow-30:crow+30, ccol-30:ccol+30] = 1
# apply mask and inverse DFT
fshift = dft_shift * mask
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

plt.subplot(131), plt.imshow(img, cmap='gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(132), plt.imshow(magnitude_spectrum, cmap='gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.subplot(133), plt.imshow(img_back, cmap='gray')
plt.title('Inverse DFT'), plt.xticks([]), plt.yticks([])
plt.show()


## 第三部分 - 掌握 imgproc 组件

### 第六章 - 图像处理

#### 6.1 & 6.2 - 线性和非线性滤波

In [19]:
# 图像处理
img = cv2.imread("./image/34_LinearImageFilter.jpg")
img_bf = cv2.imread("./image/36_BilateralFilter.png")


def on_box_filter(_):
    size_box_filter = cv2.getTrackbarPos("Box Filter", "Box Filter Result")
    
    # 方框滤波
    filter = cv2.boxFilter(img, ddepth=2, ksize=(size_box_filter+1, size_box_filter+1), anchor=(-1, -1), normalize=False) # ddepth 指定输出图像的深度，-1 代表原图深度；anchor 默认值为 (-1, -1)，即核的中心
    
    result = np.hstack([img, filter])
    cv2.imshow("Box Filter Result", result)
    
    
def on_mean_blur(_):
    size_mean_blur = cv2.getTrackbarPos("Mean Blur", "Mean Blur Result")
    
    # 均值滤波 - 均一化后的方框滤波
    filter = cv2.blur(img, ksize=(size_mean_blur+1, size_mean_blur+1), anchor=(-1, -1))
    # filter = cv2.boxFilter(img, ddepth=-1, ksize=(size_mean_blur+1, size_mean_blur+1), anchor=(-1, -1), normalize=True)
    
    psnr = cv2.PSNR(img, filter, R=255)
    print("When the size of mean blur is %d, PSNR = %f" % (size_mean_blur, psnr))
    
    result = np.hstack([img, filter])
    cv2.imshow("Mean Blur Result", result)
    
    
def on_gaussian_blur(_):
    size_gaussian_blur = cv2.getTrackbarPos("Gaussian Blur", "Gaussian Blur Result")
    
    # 高斯滤波 - 毛玻璃特效 - ksize 必须为零或正奇数
    filter = cv2.GaussianBlur(img, ksize=(size_gaussian_blur*2+1, size_gaussian_blur*2+1), sigmaX=0, sigmaY=0)

    result = np.hstack([img, filter])
    cv2.imshow("Gaussian Blur Result", result)
    

def on_median_blur(_):
    size_median_blur = cv2.getTrackbarPos("Median Blur", "Median Blur Result")
    
    # 中值滤波 - 保留边缘信息 - runtime 是均值滤波的 5 倍以上
    filter = cv2.medianBlur(img, ksize=size_median_blur*2+3)  # ksize 必须是大于 1 的奇数
    
    result = np.hstack([img, filter])
    cv2.imshow("Median Blur Result", result)


def on_bilateral_filter(_):
    size_bf = cv2.getTrackbarPos("Bilateral Filter", "Bilateral Filter Result")
    
    # 双边滤波 - 磨皮效果
    """
    d 表示在过滤过程中每个像素领域的直径；当 d > 0 时，d 指定了领域大小且于 sigmaSpace 无关，否则 d 正比于 sigmaSpace
    sigmaColor 参数值越大，表明该像素领域内有越宽广的的颜色会被混合在一起，产生较大的半相等颜色区域
    sigmaSpace 参数值越大，意味着越远的像素会相互影响，从而使更大的区域中足够相似的颜色获取相同的颜色
    """
    filter = cv2.bilateralFilter(img_bf, d=size_bf, sigmaColor=size_bf*2, sigmaSpace=size_bf/2)
    
    result = np.hstack([img_bf, filter])
    cv2.imshow("Bilateral Filter Result", result)


# === 线性滤波 ===
# 方框滤波
cv2.namedWindow("Box Filter Result")
cv2.createTrackbar("Box Filter", "Box Filter Result", 6, 50, on_box_filter)
on_box_filter(0)
# 均值滤波 - 均一化后的方框滤波
cv2.namedWindow("Mean Blur Result")
cv2.createTrackbar("Mean Blur", "Mean Blur Result", 10, 50, on_mean_blur)
on_mean_blur(0)
# 高斯滤波 - 毛玻璃特效 - ksize 必须为零或正奇数
cv2.namedWindow("Gaussian Blur Result")
cv2.createTrackbar("Gaussian Blur", "Gaussian Blur Result", 6, 50, on_gaussian_blur)
on_gaussian_blur(0)

# === 非线性滤波 ===
# 中值滤波 - 保留边缘信息 - runtime 是均值滤波的 5 倍以上
cv2.namedWindow("Median Blur Result")
cv2.createTrackbar("Median Blur", "Median Blur Result", 10, 50, on_median_blur)
on_median_blur(0)
# 双边滤波 - 磨皮效果
cv2.namedWindow("Bilateral Filter Result")
cv2.createTrackbar("Bilateral Filter", "Bilateral Filter Result", 25, 50, on_bilateral_filter)
on_bilateral_filter(0)

cv2.waitKey()
cv2.destroyAllWindows()


When the size of mean blur is 10, PSNR = 19.586833


#### 6.3 & 6.4 - 形态学滤波

In [20]:
# === 形态学滤波 === 形态学操作就是基于形状的一系列图像处理操作
# img = cv2.imread("./image/3_ImageErode.jpg")
img = cv2.imread("./image/48_Morphology.jpg")


def Open(img, element, iter):
    return cv2.dilate(cv2.erode(img, kernel=element, iterations=iter), kernel=element, iterations=iter)


def Close(img, element, iter):
    return cv2.erode(cv2.dilate(img, kernel=element, iterations=iter), kernel=element, iterations=iter)


def on_morphology(_):
    func = cv2.getTrackbarPos("Function", "Morphology Result")
    size = cv2.getTrackbarPos("Element Size", "Morphology Result")
    iter = cv2.getTrackbarPos("Iteration", "Morphology Result")
    shape = cv2.getTrackbarPos("Shape", "Morphology Result")
    
    """
    返回值为指定形状和尺寸的结构元素（内核矩阵）
    可选形状为矩形 MORPH_RECT 、交叉形 MORPH_CROSS 、椭圆形 MORPH_ELLIPSE
    交叉形的 element 形状唯一依赖于锚点的位置，而其他情况下，锚点只影响形态学运算结果的偏移
    """
    if shape == 0:
        element = cv2.getStructuringElement(
            shape=cv2.MORPH_RECT, ksize=(size+1, size+1), anchor=(-1, -1))
    elif shape == 1:
        element = cv2.getStructuringElement(
            shape=cv2.MORPH_CROSS, ksize=(size+1, size+1), anchor=(size, size))
    else:
        element = cv2.getStructuringElement(
            shape=cv2.MORPH_ELLIPSE, ksize=(size+1, size+1), anchor=(-1, -1))
    
    # 进行腐蚀或膨胀操作
    if func == 0:
        # 图像腐蚀 - 求局部最小值
        dst = cv2.erode(img, element, iterations=iter) # iteration 表示迭代使用函数的次数，其值越高，模糊程度(腐蚀程度)就越高 呈正相关关系且只能是整数，默认值为 1
    elif func == 1:
        # 图像膨胀 - 求局部最大值
        dst = cv2.dilate(img, element, iterations=iter)
    elif func == 2:
        # 开运算 - 先腐蚀后膨胀
        """
        可以用来消除小物体，在纤细点处分离物体
        并且在平滑较大物体的边界的同时不明显改变其面积
        其结果是放大了裂缝或者局部低亮度区域
        """
        # dst = Open(img, element, iter)
        dst = cv2.morphologyEx(img, cv2.MORPH_OPEN, element, iterations=iter)
    elif func == 3:
        # 闭运算 - 先膨胀后腐蚀 - 能够排除小型黑洞
        # dst = Close(img, element, iter)
        dst = cv2.morphologyEx(img, cv2.MORPH_CLOSE, element, iterations=iter)
    elif func == 4:
        # 形态学梯度 - 膨胀图与腐蚀图之差 - 可以 突出/保留 物体的边缘轮廓
        # dst = cv2.dilate(img, element, iterations=iter) - cv2.erode(img, element, iterations=iter)
        dst = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, element, iterations=iter)
    elif func == 5:
        # 顶帽(Top Hat)运算 - 原图像与开运算之差
        """
        用来分离比临近点亮一些的斑块，突出比原图轮廓周围更明亮的区域
        在一幅图像具有大幅的背景，而微小物品比较有规律的情况下，可以使用顶帽运算进行背景提取
        """
        # dst = img - Open(img, element, iter)
        dst = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, element, iterations=iter)
    elif func == 6:
        # 黑帽(Black Hat)运算 - 闭运算与原图像之差 - 用来分离比临近点暗一些的斑块，突出比原图轮廓周围更暗的区域
        # dst = Close(img, element, iter) - img
        dst = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, element, iterations=iter)

    result = np.hstack([img, dst])  # 水平排列(按列排列)的堆栈数组，注意保持维度(图像通道数)一致
    cv2.imshow("Morphology Result", result)


cv2.namedWindow("Morphology Result")
cv2.createTrackbar("Function", "Morphology Result", 0, 6, on_morphology)
cv2.createTrackbar("Element Size", "Morphology Result", 1, 50, on_morphology)
cv2.createTrackbar("Iteration", "Morphology Result", 1, 50, on_morphology)
cv2.createTrackbar("Shape", "Morphology Result", 0, 2, on_morphology)
on_morphology(0)

cv2.waitKey()
cv2.destroyAllWindows()


#### 6.5 - 漫水填充

In [21]:
# 漫水填充 - PS 魔术棒功能
img = cv2.imread("./image/50_floodFill2.jpg")
mask = np.zeros([img.shape[0]+2, img.shape[1]+2], dtype=img.dtype)  # 掩膜比需填充的图像大一圈

fill_mode = 1       # 漫水填充模式
connectivity = 4    # 表示 floodFill 函数标识符低八位的连通域值
new_mask_val = 255  # 新的重新绘制的像素值


def on_mouse(event: int, x: int, y: int, *_):
    if event is not cv2.EVENT_LBUTTONDOWN:  # 判断鼠标左键是否被按下
        return

    loDiff = cv2.getTrackbarPos("loDiff", "Result")
    upDiff = cv2.getTrackbarPos("upDiff", "Result")
    fill_mode = cv2.getTrackbarPos("fill_mode", "Result")
    flag_cnt = cv2.getTrackbarPos("connectivity", "Result")
    
    # 空范围的漫水填充，此值设为 0
    if fill_mode == 0:
        loDiff = 0
        upDiff = 0
        
    """
    标识符的低八位 (0~7) 为 connectivity，用于控制算法的连通性 - 4连通 或 8连通
    中间八位 (8~15) 用于指定填充掩码图像的值
    高八位 (16~23) 为 FLOODFILL_FIXED_RANGE 或者 0；设置为前者时，考虑当前像素与种子像素之间的差，否则就考虑当前像素与其相邻像素的差
    """
    connectivity = 8 if flag_cnt == 1 else 4
    flags = connectivity + (255 << 8) + (cv2.FLOODFILL_FIXED_RANGE if fill_mode == 1 else 0)
    
    # 随机生成 BGR 值
    blue = np.random.randint(255)
    green = np.random.randint(255)
    red = np.random.randint(255)
    new_val = (blue, green, red)
    
    area = cv2.floodFill(img, mask,  # img 既是输入，也是输出；mask 比需填充的 img 大一圈
        seedPoint=(x, y),  # 种子像素
        newVal=new_val,    # 重新绘制的像素值
        loDiff=(loDiff, loDiff, loDiff),   # 表示当前观察像素值 与其邻域像素值或待加入的种子像素之间的 亮度或颜色的 负差最大值
        upDiff=(upDiff, upDiff, upDiff),   # 同上，为正差最大值
        flags=flags)    # int 类型的操作标识符
    cv2.imshow("mask", mask)
    cv2.imshow("Result", img)
    print("there are %d pixels had been repainted." % area[0])


cv2.namedWindow("Result")   # flags 默认为 cv2.WINDOW_AUTOSIZE，即窗口使用原图大小，且不可改变
cv2.namedWindow("mask", flags=cv2.WINDOW_NORMAL)  # 表明窗口可以被随意拖动而改变大小

# Maximal lower brightness/color difference 负差最大值
cv2.createTrackbar("loDiff", "Result", 20, 255, lambda x: x)    # 不需要 on_change 时可以随意调用一个无意义函数
# Maximal upper brightness/color difference 正差最大值
cv2.createTrackbar("upDiff", "Result", 20, 255, lambda x: x)
cv2.createTrackbar("fill_mode", "Result", 1, 2, lambda x: x)
cv2.createTrackbar("connectivity", "Result", 0, 1, lambda x: x) # 0 -> 4 连通，1 -> 8 连通

cv2.setMouseCallback("Result", on_mouse, 0)

while True:
    cv2.imshow("mask", mask)
    cv2.imshow("Result", img)
    
    key = chr(cv2.waitKey())
    
    if key == 'q':
        print("the program has exited")
        cv2.destroyAllWindows()
        break
    
    if key == 'r':
        print("Restore the original image")
        img = cv2.imread("./image/50_floodFill2.jpg")
        mask *= 0
        cv2.imshow("mask", mask)
        cv2.imshow("Result", img)


there are 3127 pixels had been repainted.
there are 575 pixels had been repainted.
there are 2027 pixels had been repainted.
there are 1257 pixels had been repainted.
Restore the original image
there are 3674 pixels had been repainted.
there are 1631 pixels had been repainted.
there are 4782 pixels had been repainted.
the program has exited


#### 6.6 - 图像金字塔与图片尺寸缩放

=== 图像金字塔 pyrUp() pyrDown() ===

高斯金字塔 - 普通的上下采样 - 常应用于图像分割，先低分辨率快速初始分割，再逐层提高分辨率优化

拉普拉斯金字塔 - 原图像减去先缩小后放大的图像 - 可以减少信息丢失

![img](image_host/2022-01-13-19-54-57.png)

其公式如下：

$$ L_{i} = G_{i} - Up(G_{i+1}) \otimes kernel $$

其中 $G_{i}$ 表示第 $i$ 层的高斯金字塔图像

pyrUp() pyrDown() 与 resize() 功能类似，但不能选择插值方式

In [22]:
# === 图片尺寸缩放 resize() ===
"""
interpolation 用于指定插值方式，可选的有：
- INTER_NEAREST     最近邻插值
- INTER_LINEAR      线性插值（默认值）- 推荐用于放大图像，速度较快
- INTER_AREA        区域插值 - 利用像素区域关系的重采样插值 - 推荐用于缩小图像
- INTER_CUBIC       三次样条插值 - 超过 4X4 像素邻域内的双三次插值 - 可用于放大图像，但速度较慢
- INTER_LANCZOS4    Lanczos 插值 - 超过 8X8 像素邻域的 Lanczos 插值
"""
img = cv2.imread("./image/54_PyrAndResize.jpg")
dst_shrink = cv2.resize(img, (img.shape[0]//2, img.shape[1]//2), interpolation=cv2.INTER_AREA)
dst_enlarge = cv2.resize(img, (img.shape[0]*2, img.shape[1]*2), interpolation=cv2.INTER_LINEAR)

cv2.imshow("Original image", img)
cv2.imshow("Shrink image", dst_shrink)
cv2.imshow("Enlarge image", dst_enlarge)

cv2.waitKey()
cv2.destroyAllWindows()


In [23]:
# resize() 对比不同插值方式
img = cv2.imread("./image/54_PyrAndResize.jpg")

modes = [cv2.INTER_LINEAR, cv2.INTER_CUBIC]
names = ["linear", "cubic"]
times = 5   # 连续缩放次数

for index, mode in enumerate(modes):
    for _ in range(times):  # 先用 区域插值 连续缩小 5 次
        img = cv2.resize(img, (img.shape[0]//2, img.shape[1]//2), interpolation=cv2.INTER_AREA)

    time = cv2.getTickCount()
    for _ in range(times):  # 再分别用 线性插值和三次样条插值 连续放大 5 次
        img = cv2.resize(img, (img.shape[0]*2, img.shape[1]*2), interpolation=mode)
    time = (cv2.getTickCount() - time) / cv2.getTickFrequency()
    print("the %s interpolation takes %f s" % (names[index], time))

    if mode == cv2.INTER_LINEAR:    # 分别记录结果
        dst_linear = img
    else:
        dst_cubic = img

    img = cv2.imread("./image/54_PyrAndResize.jpg") # 重置回原图

"""
Terminal output is as follows:
    the linear interpolation takes 0.000135 s
    the cubic interpolation takes 0.000326 s
    
可见，三次样条插值不仅效率低于线性插值，
多次缩放后的结果还会出现网格化，
而线性插值却有雾化的朦胧美
"""
result = np.hstack([img, dst_linear, dst_cubic])
cv2.imshow("Result", result)
cv2.waitKey()
cv2.destroyAllWindows()


the linear interpolation takes 0.000140 s
the cubic interpolation takes 0.030029 s


#### 6.7 - 阈值化

![img](image_host/2022-01-13-19-57-56.png)

In [24]:
# 基本阈值操作
## 固定阈值操作 Threshold() - 像素级分割
"""
THRESH_BINARY       二进制阈值 - 二值阈值化
THRESH_BINARY_INV   反二进制阈值 - 反向二值阈值化并反转
THRESH_TRUNC        截断阈值 - 截断阈值化
THRESH_TOZERO       反阈值化为 0 - 超过阈值被置为 0
THRESH_TOZERO_INV   阈值化为 0 - 低于阈值被置为 0
"""
## 自适应阈值操作 adaptiveThreshold() - patch级阈值化

img = cv2.imread("./image/55_threshold.jpg")
modes = [cv2.THRESH_BINARY, cv2.THRESH_BINARY_INV,
         cv2.THRESH_TRUNC, cv2.THRESH_TOZERO, cv2.THRESH_TOZERO_INV]


def on_threshold(_):
    index = cv2.getTrackbarPos("Mode", "Result")
    thresh = cv2.getTrackbarPos("Threshold", "Result")

    # 返回 double 类型的阈值和处理后的图像
    thre, dst = cv2.threshold(img, thresh, maxval=255, type=modes[index])
    cv2.imshow("Result", dst)


cv2.namedWindow("Result", cv2.WINDOW_NORMAL)
cv2.createTrackbar("Mode", "Result", 0, len(modes)-1, on_threshold)
cv2.createTrackbar("Threshold", "Result", 0, 255, on_threshold)
on_threshold(0)

cv2.waitKey()
cv2.destroyAllWindows()


### 第七章 - 图像变换

#### 7.1 - 边缘检测

##### 7.1.2 - Canny 算子

In [25]:
# Canny 算子
img = cv2.imread("./image/56_canny.jpg")
WIN_NAME = "Canny edge detection"
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # 转为灰度图

"""
主要评价标准：低错误率、高定位性、最小响应
步骤：
- 消除噪声 - 高斯平滑滤波器
- 计算梯度幅值和方向 - Sobel 滤波器
- 非极大值抑制 - 排除 非边缘像素，仅仅保留了一些细线条（候选边缘）
- 滞后阈值
    滞后阈值需要 高阈值 和 低阈值：
    若某一像素位置的幅值 > 高阈值，该像素被保留为边缘像素
    若某一像素位置的幅值 < 低阈值，该像素被排除
    若低阈值 < 某一像素位置的幅值 < 高阈值，该像素仅仅在连接到一个高于高阈值的像素时被保留
"""
img_gray_blur = cv2.blur(img_gray, (3, 3))  # 降噪


def on_canny(_):
    thre = cv2.getTrackbarPos("thre", WIN_NAME)
    aper = cv2.getTrackbarPos("aper", WIN_NAME)
    is_l2 = False if cv2.getTrackbarPos("is_l2", WIN_NAME) == 0 else True
    
    """
    edges           输出的边缘图。需要和原图有相同的尺寸和类型

    threshold1 
    threshold2      滞后性阈值，选择两者中的较小值用于边缘连接，
                    而较大的值用来控制强边缘的初始段，
                    推荐的高低阈值比在 2:1 到 3:1 之间
    apertureSize    表示应用 Sobel 算子的孔径大小，
                    默认值为 3，必须取 3 5 7
    L2gradient      计算图像梯度幅值的标识，默认值为 False
    """
    img_edge = cv2.Canny(img_gray_blur, threshold1=thre, threshold2=thre*3, apertureSize=aper*2+3, L2gradient=is_l2)
    
    cv2.imshow(WIN_NAME, cv2.copyTo(src=img, mask=img_edge))


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("thre", WIN_NAME, 3, 50, on_canny)
cv2.createTrackbar("aper", WIN_NAME, 0, 2, on_canny)
cv2.createTrackbar("is_l2", WIN_NAME, 0, 1, on_canny)
on_canny(0)

cv2.waitKey()
cv2.destroyAllWindows()


##### 7.1.3 - Sobel 算子

In [26]:
# Sobel 算子
"""
离散微分算子 - 结合了高斯平滑和微分求导，因此结果更具抗噪性
用来计算图像灰度函数的近似梯度，在图像的任何一点使用此算子，都将会产生对应的梯度矢量或其法矢量
"""
img = cv2.imread("./image/57_sobel.jpg")
WIN_NAME = "Sobel Operator"


def on_sobel(_):
    # dx >= 0 && dy >= 0 && dx+dy > 0 in function
    vdx = cv2.getTrackbarPos("value of dx", WIN_NAME)   # x 方向上的差分阶数
    vdy = cv2.getTrackbarPos("value of dy", WIN_NAME)   # y 方向上的差分阶数
    mode = cv2.getTrackbarPos("mode", WIN_NAME) # 0 -> 整体方向；1 -> X 方向梯度；2 -> Y 方向梯度S
    ksize = cv2.getTrackbarPos("ksize", WIN_NAME) * 2 + 1  # Sobel 核的大小，必须取 1 3 5 7，且 ksize > order
    scale = cv2.getTrackbarPos("scale", WIN_NAME) * 0.1   # 计算导数值时可选的缩放因子，默认为 1
    delta = cv2.getTrackbarPos("delta", WIN_NAME) * 0.1   # 表示在结果存储到 dst 之前添加到结果的可选增量值，默认为 0
    
    vdx = vdx if mode != 2 else 0
    vdx = vdx if vdx < ksize else (ksize - 1)
    vdy = vdy if mode != 1 else 0
    vdy = vdy if vdy < ksize else (ksize - 1)
    if vdx == 0 and vdy == 0:
        vdx = 1
    """
    ddepth 输出图像的深度
    - 若 src.depth() = CV_8U，          取 ddepth = -1/CV_16S/CV_32F/CV_64F
    - 若 src.depth() = CV_16U/CV_16S，  取 ddepth = -1/CV_32F/CV_64F
    - 若 src.depth() = CV_32F，         取 ddepth = -1/CV_32F/CV_64F
    - 若 src.depth() = CV_64F，         取 ddepth = -1/CV_64F
    """
    img_sobel = cv2.Sobel(img, ddepth=int(-1/cv2.CV_16S), dx=vdx, dy=vdy, ksize=ksize, scale=scale, delta=delta)
    img_sobel_abs = cv2.convertScaleAbs(img_sobel)
    cv2.imshow(WIN_NAME, img_sobel_abs)


cv2.namedWindow(WIN_NAME, 0)
cv2.createTrackbar("value of dx", WIN_NAME, 1, 50, on_sobel)
cv2.createTrackbar("value of dy", WIN_NAME, 1, 50, on_sobel)
cv2.createTrackbar("mode", WIN_NAME, 0, 2, on_sobel)
cv2.createTrackbar("ksize", WIN_NAME, 0, 2, on_sobel)
cv2.createTrackbar("scale", WIN_NAME, 10, 100, on_sobel)
cv2.createTrackbar("delta", WIN_NAME, 10, 100, on_sobel)
on_sobel(0)

cv2.waitKey()
cv2.destroyAllWindows()


##### 7.1.5 - scharr 滤波器

In [27]:
# scharr 滤波器
"""
计算图像差分
当内核大小为 3 时，由于 Sobel 算子只是求取了导数的近似值，其内核可能产生比较明显的误差。
为解决这一问题，OpenCV 提供了 Scharr 函数，仅作用于大小为 3 的内核。
Scharr 函数与 Sobel 函数一样快，但结果更加精确。
除了没有 ksize 之外，其他参数变量基本一致
"""
img = cv2.imread("./image/59_Scharr.jpg")
WIN_NAME = "Scharr Filter"


def on_scharr(_):
    # dx >= 0 && dy >= 0 && dx+dy == 1 in function
    vdx = cv2.getTrackbarPos("value of dx", WIN_NAME)
    scale = cv2.getTrackbarPos("scale", WIN_NAME) * 0.1
    delta = cv2.getTrackbarPos("delta", WIN_NAME) * 0.1
    
    sobel = cv2.Sobel(img, ddepth=int(-1/cv2.CV_16S),
        dx=vdx, dy=1-vdx, ksize=3, scale=scale, delta=delta)
    sobel = cv2.convertScaleAbs(sobel)
    scharr = cv2.Scharr(img, ddepth=int(-1/cv2.CV_16S),
        dx=vdx, dy=1-vdx, scale=scale, delta=delta)
    scharr = cv2.convertScaleAbs(scharr)
    result = np.hstack([sobel, scharr])
    cv2.imshow(WIN_NAME, result)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("value of dx", WIN_NAME, 1, 1, on_scharr)
cv2.createTrackbar("scale", WIN_NAME, 10, 100, on_scharr)
cv2.createTrackbar("delta", WIN_NAME, 10, 100, on_scharr)
on_scharr(0)


cv2.waitKey()
cv2.destroyAllWindows()

##### 7.1.4 - Laplacian 算子

In [28]:
# Laplacian 算子
"""
Laplacian 算子是 n 维欧几里得空间中的一个二阶微分算子，定义为梯度 grad 的散度 div。
二阶导数可以用来进行检测边缘。
因为图像是“二维”，需要在两个方向进行求导，使用 Laplacian 算子将会使求导过程变得简单。
由于使用了图像梯度，所以其内部代码调用了 Sobel 算子。
"""
img = cv2.imread("./image/58_Laplacian.jpg")
WIN_NAME = "Laplacian Operator"


def on_laplacian(_):
    size = cv2.getTrackbarPos("size", WIN_NAME) # 高斯模糊核大小
    ksize = cv2.getTrackbarPos("ksize", WIN_NAME) * 2 + 1   # 计算二阶导数的滤波器孔径大小，必须为正奇数，默认值为 1
    scale = cv2.getTrackbarPos("scale", WIN_NAME) * 0.1   # 计算拉普拉斯值的时候可选的比例因子，默认值为 1
    delta = cv2.getTrackbarPos("delta", WIN_NAME) * 0.1   # 表示在结果存储到目标图之前添加到结果的可选增量值，默认为 0
    
    # 使用高斯滤波消除噪声
    filter = cv2.GaussianBlur(img, ksize=(size*2+1, size*2+1), sigmaX=0, sigmaY=0)
    img_gray = cv2.cvtColor(filter, cv2.COLOR_RGB2GRAY) # 转换为灰度图
    img_laplacian = cv2.Laplacian(img_gray, ddepth=cv2.CV_16S, ksize=ksize, scale=scale, delta=delta)
    img_abs = cv2.convertScaleAbs(img_laplacian)    # 计算绝对值，并将结果转换成 8 位
    
    cv2.imshow(WIN_NAME, img_abs)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("size", WIN_NAME, 1, 10, on_laplacian)
cv2.createTrackbar("ksize", WIN_NAME, 1, 3, on_laplacian)
cv2.createTrackbar("scale", WIN_NAME, 10, 100, on_laplacian)
cv2.createTrackbar("delta", WIN_NAME, 0, 100, on_laplacian)
on_laplacian(0)


cv2.waitKey()
cv2.destroyAllWindows()


#### 7.2 - 霍夫变换

霍夫变换是识别图像几何形状的基本方法之一，用于快速准确地检测出图中的直线或(椭)圆。在使用霍夫变换之前，首先要对图像进行边缘检测。

它是图像处理中的一种特征提取技术，该过程在一个参数空间中通过计算累计结果的局部最大值得到一个符合该特定形状的集合，作为霍夫变换的结果。

霍夫变换将 在一个空间中具有相同形状的曲线或直线 映射到 另一个坐标空间的一个点上 形成峰值，从而把检测任意形状的问题转化为统计峰值问题。

##### 7.2.3 - 霍夫线变换的原理

OpenCV 中通过两个函数来支持三种不同的霍夫线变换：

- HoughLines()
  - 标准霍夫变换 Standard Hough Transform, SHT
  - 多尺度霍夫变换 Multi-Scale Hough Transform, MSHT
    - SHT 在多尺度下的一个变种
- HoughLinesP()
  - 累计概率霍夫变换 Progressive Probabilistic Hough Transform, PPHT
    - SHT 的一个改进
    - 在一定范围内进行霍夫变换，计算单独线段的方向以及范围，从而减少计算量、缩短计算时间，执行效率更高

霍夫线变换的原理：

1) 采用极坐标系表示直线。则直线的表达式可为：

$$ y = (-\frac{cos\theta}{sin\theta}) x + (\frac{r}{sin\theta}) $$

化简得到：

$$ r = x \cdot cos\theta + y \cdot sin\theta $$

2) 对于点 $(x_{0}, y_{0})$，可以将通过这个点的一族直线统一定义为：

$$ r_{\theta} = x_{0} \cdot cos\theta + y_{0} \cdot sin\theta $$

这意味着每一对 $(r_{\theta}, \theta)$ 代表一条通过点 $(x_{0}, y_{0})$ 的直线

3) 如果对于一个给定点 $(x_{0}, y_{0})$，在极坐标系上，对极径极角平面绘出所有通过它的直线，将得到一条正弦曲线。

4) 对图像中所有点进行上述操作，如果两个不同点所对应的曲线在平面 $\theta-r$ 相交，则意味着它们通过同一条直线

<center>
    <img src="image_host/2022-01-31-12-05-45.png" alt=""></img>
    <div></div>
</center>

5) 通过设置曲线数量阈值来定义多少条曲线相交于一点，才认为检测到了一条直线。

In [29]:
# 标准霍夫变换 SHT & 累计概率霍夫变换 PPHT
img = cv2.imread("./image/61_HoughLines.jpg")
WIN_NAME = "Hough Lines"
gray = cv2.Canny(img, threshold1=50, threshold2=200, edges=3)


def on_hough_lines(_):
    mode = cv2.getTrackbarPos("mode", WIN_NAME)
    thres = cv2.getTrackbarPos("thres", WIN_NAME)
    srn = cv2.getTrackbarPos("srn", WIN_NAME)
    stn = cv2.getTrackbarPos("stn", WIN_NAME)
    mll = cv2.getTrackbarPos("minLineLength", WIN_NAME)
    mlg = cv2.getTrackbarPos("maxLineGap", WIN_NAME)
    if srn == 0 or stn == 0:
        srn = stn = 0
    
    if mode == 0:   # SHT
        """
        lines:      数组，输出检测到的线条，每条线由矢量 (ρ, θ) 表示
        rho:        double 类型，以像素为单位的距离精度
        theta:      double 类型，以弧度为单位的角度精度
        threshold:  int 类型，累加平面的阈值参数，即识别某部分为图中的一条直线时它在累加平面中必须达到的值。
                    大于阈值的线段才可以被检测通过并返回到结果中。
        srn:        double 类型，默认为 0。对于多尺度霍夫变换，表示 rho 的除数距离，
                    粗略的累加器进步尺寸直接是 rho，而精确的为 rho/srn
        stn:        double 类型，默认为 0。对于多尺度霍夫变换，表示 theta 的除数距离，

        如果 srn 和 stn 同时为 0，就表示用标准霍夫变换，否则这两个参数都应该为正数
        """
        lines = cv2.HoughLines(gray, rho=1, theta=np.pi/180, threshold=thres, srn=srn, stn=stn)

        bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        if lines is not None:
            for i in range(lines.shape[0]): # 依次在图中绘制出每条线段
                rho = lines[i][0][0]
                theta = lines[i][0][1]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                pt1 = (int(round(x0 + 1000*(-b))), int(round(y0 + 1000*(a))))
                pt2 = (int(round(x0 - 1000*(-b))), int(round(y0 - 1000*(a))))
                cv2.line(bgr, pt1, pt2, color=(55, 100, 195), thickness=1, lineType=cv2.LINE_AA)
                
    else:
        """
        lines:          输出被检测线段的起点和终点的坐标
        minLineLength:  double 类型，默认为 0。表示最低线段的长度，否则不显示
        maxLineGap:     double 类型，默认为 0。允许将同一行点与点之间连接起来的最大距离
        """
        lines = cv2.HoughLinesP(gray, rho=1, theta=np.pi/180, threshold=thres, minLineLength=mll, maxLineGap=mlg) # PPHT

        bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        if lines is not None:
            for i in range(lines.shape[0]):
                start = (lines[i][0][0], lines[i][0][1])
                end = (lines[i][0][2], lines[i][0][3])
                cv2.line(bgr, pt1=start, pt2=end, color=(186, 88, 255), thickness=1, lineType=cv2.LINE_AA)
    
    result = np.hstack([img, bgr])
    cv2.imshow(WIN_NAME, result)


cv2.namedWindow(WIN_NAME, 0)
cv2.createTrackbar("mode", WIN_NAME, 1, 1, on_hough_lines)
cv2.createTrackbar("thres", WIN_NAME, 150, 1000, on_hough_lines)
cv2.createTrackbar("srn", WIN_NAME, 0, 50, on_hough_lines)
cv2.createTrackbar("stn", WIN_NAME, 0, 50, on_hough_lines)
cv2.createTrackbar("minLineLength", WIN_NAME, 50, 50, on_hough_lines)
cv2.createTrackbar("maxLineGap", WIN_NAME, 10, 50, on_hough_lines)
on_hough_lines(0)

cv2.waitKey()
cv2.destroyAllWindows()

##### 7.2.6 - 霍夫圆变换

直线可以由 极径、极角 $(r, \theta)$ 来表示，而圆则是由 圆心和半径 $(x_{center}, y_{center}, r)$ 来唯一确定。常常通过“霍夫梯度法”来解决圆变换问题。

##### 7.2.7 - 霍夫梯度法原理

1. 首先对图像应用边缘检测，比如用 canny 边缘检测。
2. 然后，对边缘图像中的每一个非零点，考虑其局部梯度，即用 Sobel() 函数计算x和y方向的 Sobel 一阶导数得到梯度。
3. 利用得到的梯度，由斜率指定的直线上的每一个点都在累加器中被累加，这里的斜率是从一个指定的最小值到指定的最大值的距离。
4. 同时，标记边缘图像中每一个非 0 像素的位置。
5. 然后从二维累加器中这些点中选择候选的中心，这些中心都大于给定阈值并且大于其所有近邻。这些候选的中心按照累加值降序排列，以便于最支持像素的中心首先出现。
6. 接下来对每一个中心，考虑所有的非 0 像素。
7. 这些像素按照其与中心的距离排序。从到最大半径的最小距离算起，选择非 0 像素最支持的一条半径。
8. 如果一个中心受到边缘图像非 0 像素最充分的支持，并且到前期被选择的中心有足够的距离，那么它就会被保留下来。

- 优点
  - 帮助解决三维累加器中会产生许多稀疏分布的噪声，并使结果不稳定的问题
- 缺点
  - Sobel 导数所求取的局部梯度并不稳定
  - 边缘图像的所有非 0 像素都会成为候选，如果阈值偏低，将会消耗大量计算时间
  - 每个中心只能选择一个圆，无法解决同心圆问题


In [30]:
# 霍夫圆变换
img = cv2.imread("./image/63_HoughCircles.jpg")
WIN_NAME = "Hough Circle"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(9, 9), sigmaX=2, sigmaY=2)


def on_hough_circle(_):
    dp = cv2.getTrackbarPos("dp", WIN_NAME) * 0.1 + 0.1
    md = cv2.getTrackbarPos("minDist", WIN_NAME) + 1
    p1 = cv2.getTrackbarPos("param1", WIN_NAME) + 1
    p2 = cv2.getTrackbarPos("param2", WIN_NAME) + 1
    minR = cv2.getTrackbarPos("minRadius", WIN_NAME)
    maxR = cv2.getTrackbarPos("maxRadius", WIN_NAME)
    
    """
    circles:    double 类型，输出矢量数组，每个浮点矢量包含圆心坐标和半径
    method:     使用的检测方法，OpenCV 目前只支持霍夫梯度法，即 HOUGH_GRADIENT
    dp:         double 类型，检测圆心的累加器图像和输入图像的分辨率之比的倒数
    minDist:    double 类型，霍夫变换所检测到的圆的圆心之间的最小距离
    param1:     double 类型，默认为 100。method 的对应参数，表示传递给边缘检测算子的高阈值，低阈值为高阈值的一半
    param2:     double 类型，默认为 100。method 的对应参数，表示在检测阶段圆心的累加器阈值。值越小，越可以检测到更多不存在的圆；值越大，检测到的圆就越接近完美圆形。
    minRadius:  int 类型，默认为 0。表示圆半径的最小值
    maxRadius:  int 类型，默认为 0。表示圆半径的最大值。
                如果 <= 0，则使用最大图像尺寸。如果 < 0，HOUGH_GRADIENT 返回中心而不找到半径
    """
    circles = cv2.HoughCircles(
        gray, method=cv2.HOUGH_GRADIENT, dp=dp, minDist=md, 
        param1=p1, param2=p2, minRadius=minR, maxRadius=maxR)
    
    bgr = img.copy()
    if circles is not None:
        for i in range(circles.shape[1]):
            center = (int(round(circles[0][i][0])), int(round(circles[0][i][1])))
            radius = int(round(circles[0][i][2]))
            cv2.circle(bgr, center=center, radius=radius, color=(255, 0, 0), thickness=3, lineType=8, shift=0)
    cv2.imshow(WIN_NAME, bgr)


cv2.namedWindow(WIN_NAME, 0)
cv2.createTrackbar("dp", WIN_NAME, 14, 20, on_hough_circle)
cv2.createTrackbar("minDist", WIN_NAME, 9, 100, on_hough_circle)
cv2.createTrackbar("param1", WIN_NAME, 199, 500, on_hough_circle)
cv2.createTrackbar("param2", WIN_NAME, 99, 500, on_hough_circle)
cv2.createTrackbar("minRadius", WIN_NAME, 0, 10, on_hough_circle)
cv2.createTrackbar("maxRadius", WIN_NAME, 0, 10, on_hough_circle)
on_hough_circle(0)

cv2.waitKey()
cv2.destroyAllWindows()

#### 7.3 - 重映射 - remap()

重映射是指将一幅图像中某位置的像素放置到另一个图片指定位置的过程，例如图片镜像

In [31]:
img = cv2.imread("./image/65_remap.jpg")
print(img.shape)
rows = img.shape[0]
cols = img.shape[1]

# 左右对称
"""
[[5. 4. 3. 2. 1.]
 [5. 4. 3. 2. 1.]
 [5. 4. 3. 2. 1.]]
[[0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]]
"""
hor_x = np.zeros((rows, cols), dtype=np.float32)
hor_y = np.zeros((rows, cols), dtype=np.float32)
for i in range(rows):
    hor_x[i, :] = [cols-x for x in range(cols)]
for j in range(cols):
    hor_y[:, j] = [y for y in range(rows)]

# 上下对称
"""
[[0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]]
[[3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1.]]
"""
ver_x = np.zeros((rows, cols), dtype=np.float32)
ver_y = np.zeros((rows, cols), dtype=np.float32)
for i in range(rows):
    ver_x[i, :] = [x for x in range(cols)]
for j in range(cols):
    ver_y[:, j] = [rows-y for y in range(rows)]

# 中心对称
"""
[[5. 4. 3. 2. 1.]
 [5. 4. 3. 2. 1.]
 [5. 4. 3. 2. 1.]]
[[3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1.]]
"""
cen_x = np.zeros((rows, cols), dtype=np.float32)
cen_y = np.zeros((rows, cols), dtype=np.float32)
for i in range(rows):
    cen_x[i, :] = [cols-x for x in range(cols)]
for j in range(cols):
    cen_y[:, j] = [rows-y for y in range(rows)]

# 尺度缩放
"""
[[0. 0. 0. 0. 0.]
 [0. 0. 2. 4. 0.]
 [0. 0. 2. 4. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 3. 3. 0.]]
"""
scale_x = np.zeros((rows, cols), dtype=np.float32)
scale_y = np.zeros((rows, cols), dtype=np.float32)
for i in range(rows):
    for j in range(cols):
        if cols*0.25 < j < cols*0.75 and rows*0.25 < i < rows*0.75:
            scale_x[i][j] = 2*(j-cols*0.25)+0.5
            scale_y[i][j] = 2*(i-rows*0.25)+0.5
        else:
            scale_x[i][j] = 0
            scale_y[i][j] = 0


def my_remap(src, map_x, map_y):
    return cv2.remap(src=src, 
        map1=map_x, map2=map_y, 
        interpolation=cv2.INTER_LINEAR, # resize() 函数有提到，五种插值方式中 INTER_AREA 不支持
        borderMode=cv2.BORDER_CONSTANT, # 边界模式，默认为 BORDER_CONSTANT，表示目标图像中 离群点(outliers) 的像素值不会被此函数修改
        borderValue=(0, 0, 0))  # 当有常数边界时使用，默认为 (0, 0, 0)


horizontal = my_remap(img, hor_x, hor_y)
vertical = my_remap(img, ver_x, ver_y)
reverse = my_remap(img, cen_x, cen_y)
scale = my_remap(reverse, scale_x, scale_y)

cv2.namedWindow("Remap", 0)
col1 = np.vstack([img, vertical])
col2 = np.vstack([horizontal, reverse])
result1 = np.hstack([col1, col2])
cv2.imshow("Remap", result1)

cv2.namedWindow("Scale", 0)
result2 = np.hstack([reverse, scale])
cv2.imshow("Scale", result2)

cv2.waitKey()
cv2.destroyAllWindows()


(1059, 2012, 3)


#### 7.4 - 仿射变换

Affine Transformation 是指一个向量空间进行一次线性变换并接上一个平移，变换为另一个向量空间的过程。它保持了二维图形的【平直性】和【平行性】。

任意一个仿射变换都能表示为乘以一个矩阵（线性变换）接着再加上一个向量（平移）的形式：

- 线性变换
  - 旋转
  - 缩放
- 向量加
  - 平移

仿射变换涉及到 warpAffine() 和 getRotationMatrix2D() 这两个函数：

- warpAffine() 实现一些简单的重映射
- getRotationMatrix2D() 获得旋转矩阵

In [32]:
# Affine Transformation
img = cv2.imread("./image/1_HelloOpenCV.jpg")
rows = img.shape[0]
cols = img.shape[1]

## 设置原图像 SouRCe 和目标图像 DeSTination 上的三组点以计算仿射变换
# 定义两个三角形
src_triangle = np.zeros((3, 2), dtype=np.float32)
dst_triangle = np.zeros((3, 2), dtype=np.float32)

src_triangle[0] = (0, 0)
src_triangle[1] = (cols - 1, 0)
src_triangle[2] = (0, rows - 1)

dst_triangle[0] = (cols*0.00, rows*0.33)
dst_triangle[1] = (cols*0.65, rows*0.35)
dst_triangle[2] = (cols*0.15, rows*0.60)

# 求得仿射变换
map_warp = cv2.getAffineTransform(src_triangle, dst_triangle)

# 对原图像应用刚刚求得的仿射变换
dst_warp = cv2.warpAffine(img, map_warp, (cols, rows))

## 对图像进行缩放后再旋转
# 计算绕图像中点顺时针旋转 30°、缩放因子为 0.8 的旋转矩阵
center = (dst_warp.shape[1]/2, dst_warp.shape[0]/2)
angle = -30.0   # Positive values mean counter-clockwise rotation
scale = 0.8
map_rot = cv2.getRotationMatrix2D(center, angle, scale)
# 旋转已缩放后的图像
dst_warp_rot = cv2.warpAffine(dst_warp, map_rot, (cols, rows))

cv2.imshow("Affine Transformation", dst_warp_rot)

cv2.waitKey()
cv2.destroyAllWindows()

#### 7.5 - 直方图均衡化

用于扩大图像的动态范围

In [33]:
# 直方图均衡化
img = cv2.imread("./image/68_equalizeHist.jpg")
print(img.dtype)

blue, green, red = cv2.split(img)   # 对各个通道分别做直方图均衡化
blue = cv2.equalizeHist(blue)   # 只能输入 8 位单通道图像
green = cv2.equalizeHist(green)
red = cv2.equalizeHist(red)
dst = cv2.merge([blue, green, red])

result = np.hstack([img, dst])
cv2.imshow("Equalize Histogram", result)

cv2.waitKey()
cv2.destroyAllWindows()

uint8


### 第八章 - 图像轮廓与图像分割修复

#### 8.1 - 查找并绘制轮廓

In [34]:
img = cv2.imread("./image/69_findContours.jpg")
WIN_NAME = "Find and draw the contours of image"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 灰度图

## findContours() 函数可选的
# 轮廓检索模式
modes = [cv2.RETR_EXTERNAL,   # 只检测最外层的轮廓
         cv2.RETR_LIST,       # 提取所有轮廓，并且放置在 list 中。检测的轮廓不建立等级关系
         cv2.RETR_CCOMP,      # 提取所有轮廓，并且将其组织为双层结构：顶层为连通域的外围边界，次层为孔的内层边界
         cv2.RETR_TREE]       # 提取所有轮廓，并重新建立网状的轮廓结构
# 轮廓近似办法
methods = [cv2.CHAIN_APPROX_NONE,       # 获取每个轮廓的每个像素，相邻的两个点的像素位置差不超过 1
           cv2.CHAIN_APPROX_SIMPLE,     # 压缩水平、垂直、对角线方向的元素，只保留该方向的终点坐标。例如用四个点表示一个矩形
           cv2.CHAIN_APPROX_TC89_L1,    # 使用 Teh-Chin1 链逼近算法中的一个
           cv2.CHAIN_APPROX_TC89_KCOS]


def on_contours(_):
    t = cv2.getTrackbarPos("threshold", WIN_NAME)
    idx_mode = cv2.getTrackbarPos("modes index", WIN_NAME)
    idx_method = cv2.getTrackbarPos("methods index", WIN_NAME)
    tn = cv2.getTrackbarPos("thickness", WIN_NAME) - 1
    lt = (cv2.getTrackbarPos("line type", WIN_NAME) + 1) * 4
    ml = cv2.getTrackbarPos("max level", WIN_NAME)
    dx = cv2.getTrackbarPos("dx", WIN_NAME)
    dy = cv2.getTrackbarPos("dy", WIN_NAME)
    
    # _, src = cv2.threshold(gray, thresh=t, maxval=255, type=cv2.THRESH_BINARY)  # 取图片中阈值大于 t 的部分
    src = cv2.Canny(gray, threshold1=t, threshold2=t*2, apertureSize=3)
    dst = np.zeros(img.shape, img.dtype)
    
    # 寻找轮廓 - findContours()
    """
    contours    将检测到的轮廓信息以点向量形式储存
    hierarchy   可选的输出变量，包含图像的拓扑信息。
                每个 contour 对应 4 个 hierarchy 元素，
                分别表示后一个轮廓、前一个轮廓、父轮廓、内嵌轮廓的索引编号。如果没有对应值，则设为负数
    """
    contours, hierarchy = cv2.findContours(src, mode=modes[idx_mode], method=methods[idx_method])

    # 绘制轮廓 - drawContours()
    for idx in range(len(contours)):
        color = (np.random.randint(255), np.random.randint(255), np.random.randint(255))
        cv2.drawContours(dst,       # 输出图像
            contours=contours,
            contourIdx=idx,         # 轮廓绘制的指示变量。如果为负值，则绘制所有轮廓
            color=color,            # 轮廓颜色
            thickness=tn,           # 轮廓线条粗细，默认为 1；若为负值(cv2.FILLED)，则绘制在轮廓的内部
            lineType=lt,            # 线条类型，默认为 8 【8 - 八联通，4 - 四联通，LINE_AA(16) - 抗锯齿线性】
            hierarchy=hierarchy,    # 可选的层次结构信息
            maxLevel=ml,            # 用于绘制轮廓的最大等级，默认为 INTER_MAX(7)。如果为 0，则仅绘制指定的轮廓。如果为 1，该函数将绘制轮廓和所有嵌套轮廓。如果为 2，函数将绘制等高线、所有嵌套等高线、所有嵌套到嵌套的等高线，依此类推。仅当存在可用的层次结构时，才会考虑此参数。
            offset=(dx, dy))        # 指定所绘制的轮廓所需的偏移量，默认为 (0, 0)

    result = np.hstack([img, dst])
    cv2.imshow("Find and draw the contours of image", result)


cv2.namedWindow(WIN_NAME, 0)
cv2.createTrackbar("threshold", WIN_NAME, 119, 255, on_contours)
cv2.createTrackbar("modes index", WIN_NAME, 0, len(modes)-1, on_contours)
cv2.createTrackbar("methods index", WIN_NAME, 0, len(methods)-1, on_contours)
cv2.createTrackbar("thickness", WIN_NAME, 0, 20, on_contours)
cv2.createTrackbar("line type", WIN_NAME, 0, 2, on_contours)
cv2.createTrackbar("max level", WIN_NAME, 7, 7, on_contours)
cv2.createTrackbar("dx", WIN_NAME, 0, 20, on_contours)
cv2.createTrackbar("dy", WIN_NAME, 0, 20, on_contours)
on_contours(0)

cv2.waitKey()
cv2.destroyAllWindows()


#### 8.2 - 寻找物体的凸包

给定二维平面上的点集，凸包 (convex hull) 是指由最外层的点所连接构成的凸多边形。

理解物体形状或轮廓的一种比较有用的方法便是计算一个物体的凸包，然后计算其凸缺陷 (convexity defects)，许多复杂物体的特性可以被这种缺陷很好地表现出来。

![img](image_host/2022-02-24-23-46-17.png)

比如上图中 A 到 H 表示人手图的凸缺陷。这些凸缺陷提供了手及其状态的特征表现方法。

In [35]:
img = cv2.imread("./image/72_convexHull.jpg")
WIN_NAME = "Find the convex hull of stuff"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 灰度图


def on_convex_hull(_):
    t = cv2.getTrackbarPos("threshold", WIN_NAME)
    
    _, src = cv2.threshold(gray, thresh=t, maxval=255, type=cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(src, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

    hull = []
    for i in range(len(contours)):
        hull.append(cv2.convexHull(contours[i], clockwise=False))
    
    dst = np.zeros(img.shape, img.dtype)
    cv2.drawContours(dst, hull, -1, (255, 255, 255), 1, 8, hierarchy, 7, (0, 0))
    result = np.hstack([img, dst])
    cv2.imshow(WIN_NAME, result)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("threshold", WIN_NAME, 190, 255, on_convex_hull)
on_convex_hull(0)

cv2.waitKey()
cv2.destroyAllWindows()


#### 8.3 - 使用多边形将轮廓包围

##### 8.3.6/7 - 创建包围轮廓的矩形/圆形边界

In [36]:
img = np.zeros((600, 600, 3), dtype="uint8")
WIN_NAME = "create the bounding of contour"
rows = img.shape[0]
cols = img.shape[1]
count = np.random.randint(3, 103)   # 随机生成点的数量
points = np.zeros([count, 2], dtype="float32")
for i in range(count):
    # 生成随机坐标
    points[i] = [np.random.randint(cols/4, cols*3/4),
        np.random.randint(rows/4, rows*3/4)]
    # 绘制随机点
    color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
    cv2.circle(img,
        center=tuple(points[i].astype(int)),
        radius=3,
        color=color,
        thickness=cv2.FILLED,
        lineType=cv2.LINE_AA)

color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))


def on_bounding(_):
    mode = cv2.getTrackbarPos("mode", WIN_NAME)
    
    src = img.copy()
    if mode == 0:
        # 对给定的二维点集，寻找最小面积的包围矩形
        rect = cv2.minAreaRect(points)
        # 绘制出最小面积的包围矩形
        box = cv2.boxPoints(rect)
        for i in range(4):
            cv2.line(src,
                pt1=tuple(box[i].astype(int)),
                pt2=tuple(box[(i+1) % 4].astype(int)),
                color=color,
                thickness=2,
                lineType=cv2.LINE_AA)
    else:
        # 对给定的二维点集，寻找最小面积的包围圈
        center, radius = cv2.minEnclosingCircle(points)
        # 绘制出最小面积的包围圈
        for i in range(count):
            cv2.circle(src,
                center=(int(center[0]), int(center[1])),
                radius=int(radius),
                color=color,
                thickness=2,
                lineType=cv2.LINE_AA)
    
    cv2.imshow(WIN_NAME, src)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("mode", WIN_NAME, 0, 1, on_bounding)
on_bounding(0)

cv2.waitKey()
cv2.destroyAllWindows()


#### 8.4 - 图像的矩

In [38]:
img = cv2.imread("./image/76_ContourMoment.jpg")
WIN_NAME = "Moments"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.blur(gray, (3, 3))


def on_moments(_):
    t = cv2.getTrackbarPos("threshold", WIN_NAME)

    src = cv2.Canny(blur, threshold1=t, threshold2=t*2, apertureSize=3) # 检测边缘
    # 寻找轮廓
    contours, hierarchy = cv2.findContours(
        src, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

    # 计算矩
    moments = []
    for i in range(len(contours)):
        moments.append(cv2.moments(contours[i], binaryImage=False))
    # 计算中心距
    points = np.zeros([len(contours), 2], dtype="float32")
    for i in range(len(contours)):
        if moments[i]['m00'] == 0:
            points[i] = (float('inf'), float('inf'))
            continue
        points[i] = ((moments[i]['m10'] / moments[i]['m00']), (moments[i]['m01'] / moments[i]['m00']))

    # 绘制轮廓
    dst = np.zeros(img.shape, img.dtype)
    for i in range(len(contours)):
        color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
        # 绘制外层和内层轮廓
        cv2.drawContours(dst, contours, i, color, 2, 8, hierarchy, 0, (0, 0))
        # 绘制圆
        cv2.circle(dst, tuple(points[i].astype(int)), 4, color, -1, 8, 0)

    result = np.hstack([img, dst])
    cv2.imshow(WIN_NAME, result)
    cv2.imshow("canny", src)
    
    # 通过 m00 计算轮廓面积并且和 OpenCV 函数比较
    print("输出内容：面积和轮廓长度")
    for i in range(len(contours)):
        print("通过 m00 计算出轮廓", i, "的面积：", moments[i]["m00"])

        """
        contourArea()
            用于计算整个轮廓或部分轮廓的面积。
            contour     二维向量，轮廓顶点。
            oriented    面向区域标识符。
                        若为 True，该函数返回一个带符号的面积值，
                        其正负取决于轮廓的方向。默认为 False。
        arcLength()
            用于计算封闭轮廓的周长或曲线的长度
            curve       二维点集
            closed      用于指示曲线是否封闭的标识符。
                        默认为 closed，表示曲线封闭。
        """
        print("OpenCV 函数计算出的面积：%.2f，长度：%.2f" % (cv2.contourArea(contour=contours[i], oriented=True), cv2.arcLength(curve=contours[i], closed=True)))


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("threshold", WIN_NAME, 100, 255, on_moments)
on_moments(0)

cv2.waitKey()
cv2.destroyAllWindows()


输出内容：面积和轮廓长度
通过 m00 计算出轮廓 0 的面积： 2.0
OpenCV 函数计算出的面积：-2.00，长度：18.49
通过 m00 计算出轮廓 1 的面积： 5.0
OpenCV 函数计算出的面积：-5.00，长度：29.80
通过 m00 计算出轮廓 2 的面积： 11.0
OpenCV 函数计算出的面积：-11.00，长度：13.66
通过 m00 计算出轮廓 3 的面积： 8.0
OpenCV 函数计算出的面积：8.00，长度：11.31
通过 m00 计算出轮廓 4 的面积： 12.0
OpenCV 函数计算出的面积：-12.00，长度：75.25
通过 m00 计算出轮廓 5 的面积： 0.5
OpenCV 函数计算出的面积：-0.50，长度：22.24
通过 m00 计算出轮廓 6 的面积： 7.0
OpenCV 函数计算出的面积：-7.00，长度：47.11
通过 m00 计算出轮廓 7 的面积： 0.5
OpenCV 函数计算出的面积：-0.50，长度：47.90
通过 m00 计算出轮廓 8 的面积： 13.5
OpenCV 函数计算出的面积：-13.50，长度：14.24
通过 m00 计算出轮廓 9 的面积： 9.5
OpenCV 函数计算出的面积：9.50，长度：11.90
通过 m00 计算出轮廓 10 的面积： 7.0
OpenCV 函数计算出的面积：-7.00，长度：81.60
通过 m00 计算出轮廓 11 的面积： 5.0
OpenCV 函数计算出的面积：-5.00，长度：20.14
通过 m00 计算出轮廓 12 的面积： 0.5
OpenCV 函数计算出的面积：-0.50，长度：21.90
通过 m00 计算出轮廓 13 的面积： 12.0
OpenCV 函数计算出的面积：-12.00，长度：13.66
通过 m00 计算出轮廓 14 的面积： 8.5
OpenCV 函数计算出的面积：8.50，长度：11.07
通过 m00 计算出轮廓 15 的面积： 13.0
OpenCV 函数计算出的面积：-13.00，长度：34.63
通过 m00 计算出轮廓 16 的面积： 8.0
OpenCV 函数计算出的面积：8.00，长度：11.31
通过 m00 计算出轮廓 17 的面积： 1.5
OpenCV 函数计算出的面

#### 8.5 - 分水岭算法

In [39]:
img = cv2.imread("./image/77_watershed.jpg")
WIN_NAME1 = "original image"
WIN_NAME2 = "watershed"
src = img.copy()
mask = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
gray = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
mask *= 0
prev_pt = (-1, -1)
cv2.imshow(WIN_NAME1, src)


def on_mouse(event, x, y, flags, _):
    global prev_pt
    # 当鼠标不在窗口中时
    if x < 0 or x >= src.shape[1] or y < 0 or y >= src.shape[0]:
        return
    
    if event == cv2.EVENT_LBUTTONUP:
        prev_pt = (-1, -1)
    elif event == cv2.EVENT_LBUTTONDOWN:
        prev_pt = (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        point = (x, y)
        if prev_pt[1] < 0:
            prev_pt = point
        cv2.line(mask, pt1=prev_pt, pt2=point, color=(255, 255, 255), thickness=5, lineType=8, shift=0)
        cv2.line(src, pt1=prev_pt, pt2=point, color=(255, 255, 255), thickness=5, lineType=8, shift=0)
        prev_pt = point
        cv2.imshow(WIN_NAME1, src)


cv2.namedWindow(WIN_NAME1)
cv2.setMouseCallback(WIN_NAME1, on_mouse, 0)

while True:
    key = chr(cv2.waitKey())
    if key == 'q':  # 退出程序
        print("the program has exited")
        cv2.destroyAllWindows()
        break
    if key == 'r':  # 还原图像
        print("Restore the original image")
        mask *= 0
        src = img.copy()
        cv2.imshow(WIN_NAME1, src)
    if key == ' ':  # 运行
        comp_count = 0
        # 寻找轮廓
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        # 轮廓为空时
        if contours is None:
            continue
        # 拷贝掩膜
        mask_img = np.zeros(mask.shape, dtype="int32")
        mask_img *= 0
        # 绘制轮廓
        for i in range(len(contours)):
            cv2.drawContours(mask_img, contours, i, tuple([comp_count + 1] * 3), -1, 8, hierarchy, cv2.INTER_MAX)
            comp_count += 1
        # comp_count 为零时
        if comp_count == 0:
            continue
        # 生成随机颜色
        color_tab = []
        for i in range(comp_count):
            color_tab.append((np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255)))
        # 计算处理时间
        time = cv2.getTickCount()
        cv2.watershed(img, mask_img)
        time = cv2.getTickCount() - time
        print("程序运行时间为", time / cv2.getTickFrequency(), "s")
        # 双层循环，将分水岭图像遍历储存
        watershed = np.zeros([mask_img.shape[0], mask_img.shape[1], 3], dtype="uint8")
        for i in range(mask_img.shape[0]):
            for j in range(mask_img.shape[1]):
                index = mask_img[i][j]
                if index == -1:
                    watershed[i][j] = (255, 255, 255)
                elif index <= 0 or index > comp_count:
                    watershed[i][j] = (0, 0, 0)
                else:
                    watershed[i][j] = color_tab[index - 1]
        # 混合灰度图和分水岭效果图
        watershed = watershed * 0.5 + gray * 0.5
        cv2.imshow(WIN_NAME2, watershed.astype("uint8"))


程序运行时间为 0.0138116 s
the program has exited


#### 8.6 - 图像修补

In [40]:
img = cv2.imread("./image/78_inpaint.jpg")
WIN_NAME = "original image"
src = img.copy()
mask = np.zeros([src.shape[0], src.shape[1]], src.dtype)
prev_pt = (-1, -1)
cv2.imshow(WIN_NAME, src)


def on_mouse(event, x, y, flags, _):
    global prev_pt
    # 当鼠标不在窗口中时
    if x < 0 or x >= src.shape[1] or y < 0 or y >= src.shape[0]:
        return
    
    if event == cv2.EVENT_LBUTTONUP:
        prev_pt = (-1, -1)
    elif event == cv2.EVENT_LBUTTONDOWN:
        prev_pt = (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        point = (x, y)
        if prev_pt[1] < 0:
            prev_pt = point
        cv2.line(mask, pt1=prev_pt, pt2=point, color=(255, 255, 255), thickness=5, lineType=8, shift=0)
        cv2.line(src, pt1=prev_pt, pt2=point, color=(255, 255, 255), thickness=5, lineType=8, shift=0)
        prev_pt = point
        cv2.imshow(WIN_NAME, src)


cv2.namedWindow(WIN_NAME)
cv2.setMouseCallback(WIN_NAME, on_mouse, 0)

while True:
    key = chr(cv2.waitKey())
    if key == 'q':  # 退出程序
        print("the program has exited")
        cv2.destroyAllWindows()
        break
    if key == 'r':  # 还原图像
        print("Restore the original image")
        mask *= 0
        src = img.copy()
        cv2.imshow(WIN_NAME, src)
    if key == ' ':  # 运行
        inpaint = cv2.inpaint(src, mask, 3, cv2.INPAINT_TELEA)
        cv2.imshow("inpaint", inpaint)


Restore the original image
the program has exited


### 第九章 - 直方图与匹配

#### 9.2 - 直方图的计算与绘制

##### 9.2.3 - 绘制 H-S 直方图

In [41]:
img = cv2.imread("./image/81_histogram.jpg")
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

hue_bin = 30    # 色调 (hue) 直方图的 bin 数量
sat_bin = 32     # 饱和度 (saturation) 直方图的 bin 数量
hist_size = [hue_bin, sat_bin]

hue_range = [0, 180]    # 定义色调的变化范围
sat_range = [0, 256]    # 定义饱和度的变化范围
ranges = hue_range + sat_range

channels = [0, 1]   # calcHist() 函数中将计算第 0 和第 1 通道的直方图

# 计算一个或者多个阵列的直方图
hist = cv2.calcHist([hsv],
    channels=channels,      # 通道索引
    mask=None,              # 不使用掩膜
    histSize=hist_size,     # 存放每个维度的直方图尺寸的数组
    ranges=ranges,          # 每个维度数值的取值范围数组
    accumulate=False)       # 累计标识符。True 表示直方图在配置阶段会被清零。此功能主要是允许从多个阵列中计算单个直方图，或者用于在特定的时间更新直方图

# 在数组中找到全局最小值和最大值
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(hist, mask=None)

# 双层循环，绘制直方图
scale = 20  # 控制 H-V 直方图的显示大小
hist_img = np.zeros([sat_bin*scale, hue_bin*scale, 3], dtype="uint8")
for hue in range(hue_bin):
    for sat in range(sat_bin):
        bin_val = hist[hue][sat]  # 直方图 bin 的值
        intensity = round(bin_val * 255 / max_val)    # 强度
        cv2.rectangle(hist_img,
            pt1=(hue*scale, sat*scale),
            pt2=((hue+1)*scale-1, (sat+1)*scale-1),
            color=tuple([intensity]*3),
            thickness=cv2.FILLED)

cv2.imshow("Original Image", img)
cv2.imshow("Result", hist_img)
cv2.waitKey()
cv2.destroyAllWindows()


##### 9.2.4 - 计算并绘制图像一维直方图

In [42]:
img = cv2.imread("./image/81_histogram.jpg")
hist = cv2.calcHist([img], channels=[0], mask=None, histSize=[256], ranges=[0, 255], accumulate=False)
scale = 2   # 控制一维直方图的显示宽度，或者说 bin 的间距
size = 256
dst = np.zeros([size, size*scale], dtype="uint8")
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(hist, mask=None)
hpt = 0.9 * size

for i in range(256):
    bin_val = hist[i]
    real_val = int(np.round(bin_val * hpt / max_val))   # rectangle() 要求坐标的数据类型为 int，而 np.round 输出的是 float
    cv2.rectangle(dst, pt1=(i*scale, size-1), pt2=((i+1)*scale-1, size-real_val), color=(255, 255, 255))

cv2.imshow("Result", dst)
cv2.waitKey()
cv2.destroyAllWindows()


#### 9.3 - 直方图对比

In [43]:
"""
compareHist() 对比两个直方图的相似度
method:
- HISTCMP_CORREL            相关, Correlation
- HISTCMP_CHISQR            卡方, Chi-Square
- HISTCMP_INTERSECT         直方图相交, Intersection
- HISTCMP_BHATTACHARYYA     Bhattacharyya 距离
"""

img1 = cv2.imread("./image/82_compareHist_1.jpg")
img2 = cv2.imread("./image/82_compareHist_2.jpg")
img3 = cv2.imread("./image/82_compareHist_3.jpg")
img_show = np.hstack([img1, img2, img3])
cv2.imshow("image", img_show)

hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)    # 基准图
hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)    # 测试图
hsv3 = cv2.cvtColor(img3, cv2.COLOR_BGR2HSV)
# 创建包含基准图像下半部分的半身图像（HSV 格式）
hsv_half_down = hsv1[round(hsv1.shape[0]/2):hsv1.shape[0], 0:hsv1.shape[1]]

hue_bin = 50
sat_bin = 60
hist_size = [hue_bin, sat_bin]
hue_range = [0, 256]
sat_range = [0, 180]
ranges = hue_range + sat_range
channels = [0, 1]


def my_calcHist(img):
    hist = cv2.calcHist([img], channels=channels, mask=None, histSize=hist_size, ranges=ranges, accumulate=False)
    cv2.normalize(src=hist, dst=hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=-1, mask=None)
    
    return hist


hist1 = my_calcHist(hsv1)
hist2 = my_calcHist(hsv2)
hist3 = my_calcHist(hsv3)
hist_half_down = my_calcHist(hsv_half_down)

# 按顺序使用 4 种对比标准来对比直方图
methods_dict = {'Correlation': cv2.HISTCMP_CORREL,
                'Chi-Square': cv2.HISTCMP_CHISQR,
                '直方图相交': cv2.HISTCMP_INTERSECT,
                'Bhattacharyya 距离': cv2.HISTCMP_BHATTACHARYYA}
methods_idx = list(methods_dict.values())
methods_name = list(methods_dict.keys())
for i in range(len(methods_dict)):
    ch1 = cv2.compareHist(H1=hist1, H2=hist1, method=methods_idx[i])
    ch2 = cv2.compareHist(H1=hist1, H2=hist2, method=methods_idx[i])
    ch3 = cv2.compareHist(H1=hist1, H2=hist3, method=methods_idx[i])
    ch_half_down = cv2.compareHist(H1=hist1, H2=hist_half_down, method=methods_idx[i])
    
    print(methods_name[i], "方法的匹配结果为：")
    print("【基准图 - 基准图】\t %.3f" % ch1)
    print("【基准图 - 测试图 1】\t %.3f" % ch2)
    print("【基准图 - 测试图 2】\t %.3f" % ch3)
    print("【基准图 - 半身图】\t %.3f" % ch_half_down)
    print()

cv2.waitKey()
cv2.destroyAllWindows()


Correlation 方法的匹配结果为：
【基准图 - 基准图】	 1.000
【基准图 - 测试图 1】	 0.267
【基准图 - 测试图 2】	 0.130
【基准图 - 半身图】	 0.909

Chi-Square 方法的匹配结果为：
【基准图 - 基准图】	 0.000
【基准图 - 测试图 1】	 4940.944
【基准图 - 测试图 2】	 3960.965
【基准图 - 半身图】	 21.229

直方图相交 方法的匹配结果为：
【基准图 - 基准图】	 58.699
【基准图 - 测试图 1】	 7.323
【基准图 - 测试图 2】	 4.538
【基准图 - 半身图】	 33.234

Bhattacharyya 距离 方法的匹配结果为：
【基准图 - 基准图】	 0.000
【基准图 - 测试图 1】	 0.603
【基准图 - 测试图 2】	 0.663
【基准图 - 半身图】	 0.359



#### 9.4 - 反向投影

首先计算某一特征的直方图模型，然后使用模型去寻找图像中存在的该特征。

反向投影中储存的数值代表了测试图像中该像素属于特征区域的概率。

反向投影的过程是【原图像 -> 直方图 -> 反向投影】，算法步骤有点类似于直方图均衡化，只不过，直方图均衡化是将图像结果中的每个像素的值由一个地方搬到另一个新地方，而 back project 是直接取直方图中的值。以灰度图为例，某种灰度值在整幅图像中所占面积越大，其在直方图中的值越大，back project 时，其对应的像素的新值越大（越亮），反过来，某灰度值所占面积越小，其新值就越小。

![img](image_host/2022-03-10-11-03-03.png)

---

版权声明：上述部分表达转自 CSDN 博主「viewcode」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。

原文链接：https://blog.csdn.net/viewcode/article/details/8209067

In [44]:
img = cv2.imread("./image/83_calcBackProject.jpg")
WIN_NAME = "back project"
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

"""
# 通道复制 - 由输入参数复制某通道到输出参数特定的通道中
hue_img = np.zeros(hsv.shape, hsv.dtype)    # 所有矩阵必须初始化，且大小和深度与 src 相同
ch = [0, 0]     # 对指定的通道进行复制的数组索引
cv2.mixChannels([hsv], dst=[hue_img], fromTo=ch)    # 只使用 hue 通道值
"""
# 与上述代码等价，split() 和 merge() 是 mixChannels() 的一部分
hue_img, _, _ = cv2.split(hsv)


def on_bin_change(_):
    bin = cv2.getTrackbarPos("bin", WIN_NAME)   # 直方图组距
    scale = cv2.getTrackbarPos("scale", WIN_NAME)
    
    hist_size = [max(bin, 2)]
    ranges = [0, 180]
    
    # 计算直方图并归一化
    hist = cv2.calcHist([hue_img],
        channels=[0],
        mask=None,
        histSize=hist_size,
        ranges=ranges,
        accumulate=False)
    cv2.normalize(src=hist, dst=hist, alpha=0, beta=255,
        norm_type=cv2.NORM_MINMAX, 
        dtype=-1, mask=None)
    
    # 计算反向投影
    back_project = cv2.calcBackProject([hue_img],
        channels=[0],   # 需要统计的通道索引
        hist=hist,      # 输入的直方图
        ranges=ranges,  # 每一维数组的取值范围
        scale=scale)    # 输出的方向投影可选的缩放因子，默认为 1
    
    cv2.imshow("original image", img)
    cv2.imshow(WIN_NAME, back_project)
    
    # 绘制直方图
    w, h = 400, 400
    bin_w = round(w / max(bin, 2))
    hist_img = np.zeros([w, h, 3], dtype="uint8")
    for i in range(bin):
        cv2.rectangle(hist_img,
            pt1=(i*bin_w, h),
            pt2=((i+1)*bin_w, h-int(np.round(hist[i]*h/255))),
            color=(100, 123, 255),
            thickness=-1)
    
    cv2.imshow("hist", hist_img)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("bin", WIN_NAME, 30, 180, on_bin_change)
cv2.createTrackbar("scale", WIN_NAME, 1, 10, on_bin_change)
on_bin_change(0)

cv2.waitKey()
cv2.destroyAllWindows()

#### 9.5 - 模板匹配

模板匹配不是基于直方图，而是通过在输入图像上滑动图像块，对输入图像进行匹配的方法

In [45]:
img_base = cv2.imread("./image/84_matchTemplate_1.jpg")
template = cv2.imread("./image/84_matchTemplate_2.jpg")
WIN_NAME = "match"
methods = [cv2.TM_SQDIFF,   # 平方差匹配法
    cv2.TM_SQDIFF_NORMED,   # 归一化平方差匹配法
    cv2.TM_CCORR,           # 相关匹配法
    cv2.TM_CCORR_NORMED,    # 归一化相关匹配法
    cv2.TM_CCOEFF,          # 系数匹配法
    cv2.TM_CCOEFF_NORMED]   # 化相关系数匹配法

def on_match(_):
    idx = cv2.getTrackbarPos("method index", WIN_NAME)
    
    src = img_base.copy()
    # 进行匹配和标准化
    res = cv2.matchTemplate(src, templ=template, method=methods[idx], mask=None)
    cv2.normalize(res, res, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=-1, mask=None)
    
    # 通过 minMaxLoc() 定位最匹配的位置
    _, _, min_loc, max_loc = cv2.minMaxLoc(res, mask=None)
    
    # 对于 SQDIFF 和 SQDIFF_NORMED，越小的值有着越高的匹配度，而其余的方法与此相反
    if methods[idx] == cv2.TM_SQDIFF or methods[idx] == cv2.TM_SQDIFF_NORMED:
        match_loc = min_loc
    else:
        match_loc = max_loc
    
    # 绘制出矩形，并显示结果
    cv2.rectangle(src, pt1=match_loc,
        pt2=(match_loc[0]+template.shape[1], match_loc[1]+template.shape[0]),
        color=(0, 0, 255),
        thickness=2,
        lineType=8)
    cv2.rectangle(res, pt1=match_loc,
        pt2=(match_loc[0]+template.shape[1], match_loc[1]+template.shape[0]),
        color=(0, 0, 255),
        thickness=2,
        lineType=8)
    
    cv2.imshow("original image", src)
    cv2.imshow(WIN_NAME, res)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("method index", WIN_NAME, 0, len(methods)-1, on_match)
on_match(0)

cv2.waitKey()
cv2.destroyAllWindows()


## 第四部分 - 深入 feature2d 组件

### 第十章 - 角点检测

关于角点的描述：

- 一阶导数（即灰度的梯度）的局部最大值所对应的像素点
- 两条及两条以上边缘的交点
- 图像中梯度值和梯度方向的变化速率都很高的点
- 角点处的一阶导数最大，二阶导数为零，它指示了物体边缘变化不连续的方向。

角点检测也称为特征点检测。实际应用中，大多数所谓的角点检测方法检测的是拥有特定特征的图像点，而不仅仅是“角点”。

焦点检测的方法归纳：

- 基于灰度图像
  - 基于梯度
  - 基于模板
    - 主要考虑像素邻域点的灰度变化，即图像亮度的变化，将与邻域点亮度对比足够大的点定义为角点。
    - Kitchen-Rosenfeld 角点检测算法
    - Harris 角点检测算法
    - KLT 角点检测算法
    - SUSAN 角点检测算法
  - 基于模板梯度组合
- 基于二值图像
- 基于轮廓曲线

#### 10.1 - Harris 角点检测

稳定性高，尤其对 L 型角点检测精度高。

但由于采用了高斯滤波，运算速度相对较慢，角点信息有丢失和位置偏移的现象，而且角点提取有聚簇现象。

In [46]:
# img = cv2.imread("./image/85_cornerHarris.jpg", 0)
img = cv2.imread("./image/86_cornerHarris.jpg")
WIN_NAME = "Harris Corner Detection"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def on_corner_harris(_):
    t = cv2.getTrackbarPos("threshold", WIN_NAME)
    
    dst = cv2.cornerHarris(gray,
        blockSize=2,    # 表示邻域的大小
        ksize=3,        # 表示 Sobel() 算子的孔径大小
        k=0.04,         # Harris 参数
        borderType=cv2.BORDER_DEFAULT)  # 图像像素的边界模式
    _, harris = cv2.threshold(dst, thresh=1e-4, maxval=255, type=cv2.THRESH_BINARY)
    
    # 归一化与转化
    norm = np.zeros(dst.shape, dtype="float32")
    cv2.normalize(dst, norm, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32FC1, mask=None)
    scale = cv2.convertScaleAbs(norm)   # 将归一化后的图像线性变换成 uint8 数据类型
    
    # 绘制检测到的且符合阈值条件的角点
    src = img.copy()
    for i in range(norm.shape[0]):
        for j in range(norm.shape[1]):
            if norm[i][j] > t + 80:
                cv2.circle(src, center=(j, i), radius=5, color=(10, 10, 255), thickness=2, lineType=8)
                cv2.circle(scale, center=(j, i), radius=5, color=(0, 10, 255), thickness=2, lineType=8)
    
    cv2.imshow("original image", src)
    cv2.imshow(WIN_NAME, scale)
    cv2.imshow("Harris", harris)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("threshold", WIN_NAME, 30, 175, on_corner_harris)
on_corner_harris(0)

cv2.waitKey()
cv2.destroyAllWindows()

#### 10.2 - Shi-Tomasi 角点检测

In [47]:
img = cv2.imread("./image/87_goodFeaturesToTrack.jpg")
WIN_NAME = "Shi-Tomasi Corner Detection"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def on_GFT(_):
    max_corner_num = cv2.getTrackbarPos("max corner number", WIN_NAME)
    if max_corner_num <= 1:
        max_corner_num = 1
    flag = False
    if cv2.getTrackbarPos("use Harris Detector", WIN_NAME) == 1:
        flag = True
    block_size = cv2.getTrackbarPos("block size", WIN_NAME) + 1
    
    corners = cv2.goodFeaturesToTrack(gray,
        maxCorners=max_corner_num,  # 角点的最大数量
        qualityLevel=0.01,          # 角点检测可接受的最小特征值
        minDistance=10,             # 角点之间的最小距离
        mask=None,                  # 感兴趣区域
        blockSize=block_size,       # 计算导数自相关矩阵时指定的邻域范围，默认为 3
        useHarrisDetector=flag,     # 不使用 Harris 角点检测
        k=0.04)                     # 用于设置 Hessian 自相关矩阵行列式的相对权重的权重系数
    
    src = img.copy()
    for i in range(len(corners)):
        color = (np.random.randint(255), np.random.randint(255), np.random.randint(255))
        cv2.circle(src, center=(int(corners[i][0][0]), int(corners[i][0][1])),
            radius=4, color=color, thickness=-1, lineType=8)
    
    cv2.imshow(WIN_NAME, src)


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("max corner number", WIN_NAME, 33, 500, on_GFT)
cv2.createTrackbar("use Harris Detector", WIN_NAME, 0, 1, on_GFT)
cv2.createTrackbar("block size", WIN_NAME, 2, 20, on_GFT)
on_GFT(0)

cv2.waitKey()
cv2.destroyAllWindows()


#### 10.3 - 亚像素级角点检测

若我们进行图像处理的目的不是提取用于识别的特征点而是进行几何测量，这通常需要更高的精度（实数坐标值）

亚像素级角点检测的位置在摄像机标定、跟踪并重建摄像机的轨迹，或者重建被跟踪目标的三维结构时，是一个基本的测量值。

In [60]:
img = cv2.imread("./image/88_cornerSubPix.jpg")
WIN_NAME = "Sub-pixel Corner Detection"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def on_CSP(_):
    max_corner_num = cv2.getTrackbarPos("max corner number", WIN_NAME)
    if max_corner_num <= 1:
        max_corner_num = 1
    flag = False
    if cv2.getTrackbarPos("use Harris Detector", WIN_NAME) == 1:
        flag = True
    block_size = cv2.getTrackbarPos("block size", WIN_NAME) + 1
    
    corners = cv2.goodFeaturesToTrack(gray,
        maxCorners=max_corner_num,  # 角点的最大数量
        qualityLevel=0.01,          # 角点检测可接受的最小特征值
        minDistance=10,             # 角点之间的最小距离
        mask=None,                  # 感兴趣区域
        blockSize=block_size,       # 计算导数自相关矩阵时指定的邻域范围，默认为 3
        useHarrisDetector=flag,     # 不使用 Harris 角点检测
        k=0.04)                     # 用于设置 Hessian 自相关矩阵行列式的相对权重的权重系数
    
    src = img.copy()
    for i in range(len(corners)):
        color = (np.random.randint(255), np.random.randint(255), np.random.randint(255))
        cv2.circle(src, center=(int(corners[i][0][0]), int(corners[i][0][1])),
            radius=4, color=color, thickness=-1, lineType=8)
    
    cv2.imshow(WIN_NAME, src)
    
    # 计算出亚像素角点位置
    cv2.cornerSubPix(gray,
        corners=corners,
        winSize=(5, 5),
        zeroZone=(-1, -1),
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 40, 0.001))
    print(corners.shape)
    
    # 输出角点信息
    for i in range(len(corners)):
        print("精确角点坐标【" + str(i) + "】\t(" + str(corners[i][0][0]) + "," + str(corners[i][0][1]) + ")")


cv2.namedWindow(WIN_NAME)
cv2.createTrackbar("max corner number", WIN_NAME, 33, 500, on_CSP)
cv2.createTrackbar("use Harris Detector", WIN_NAME, 0, 1, on_CSP)
cv2.createTrackbar("block size", WIN_NAME, 2, 20, on_CSP)
on_CSP(0)

cv2.waitKey()
cv2.destroyAllWindows()


(33, 1, 2)
精确角点坐标【0】	(360.6143,594.07764)
精确角点坐标【1】	(103.15959,583.7792)
精确角点坐标【2】	(448.3073,591.67004)
精确角点坐标【3】	(230.0,576.0)
精确角点坐标【4】	(420.0,631.0)
精确角点坐标【5】	(183.23918,581.2203)
精确角点坐标【6】	(144.0,339.0)
精确角点坐标【7】	(144.0,216.0)
精确角点坐标【8】	(144.0,172.0)
精确角点坐标【9】	(304.0,309.0)
精确角点坐标【10】	(154.69167,367.2343)
精确角点坐标【11】	(304.0,288.0)
精确角点坐标【12】	(144.0,127.0)
精确角点坐标【13】	(321.0,483.0)
精确角点坐标【14】	(331.0,514.0)
精确角点坐标【15】	(145.0,231.0)
精确角点坐标【16】	(144.0,138.0)
精确角点坐标【17】	(62.547638,534.58203)
精确角点坐标【18】	(144.0,205.0)
精确角点坐标【19】	(327.0,501.0)
精确角点坐标【20】	(143.28537,376.61914)
精确角点坐标【21】	(162.77371,585.33075)
精确角点坐标【22】	(112.84287,590.2583)
精确角点坐标【23】	(299.0,348.0)
精确角点坐标【24】	(254.15288,536.3996)
精确角点坐标【25】	(310.0,449.0)
精确角点坐标【26】	(314.0,462.0)
精确角点坐标【27】	(296.0,237.0)
精确角点坐标【28】	(145.0,114.0)
精确角点坐标【29】	(299.0,255.0)
精确角点坐标【30】	(143.0,257.0)
精确角点坐标【31】	(302.0,274.0)
精确角点坐标【32】	(311.76083,257.8652)


### 第十一章 - 特征检测与匹配

#### 11.1 - SURF 特征点检测

Speeded Up Robust Features，加速版的具有鲁棒性的特征算法

由 Bay 在 2006 年首次提出，是尺度不变特征变换算法（SIFT）的加速版，在多幅图片下具有更好的稳定性，最大的区别在于采用了 harr 特征以及积分图像的概念。

---

学海无涯，未完待续 ...